# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 2
	2. Incremental guided retraining starting from the original model using the new adversarial inputs and original training set.

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
pwd

'C:\\Users\\fjdur\\Documents\\upc-july\\notebooks'

In [4]:
cd C:/Users/fjdur/Documents/utils//

C:\Users\fjdur\Documents\utils


In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 54D3-6FC8

 Directory of C:\Users\fjdur\Documents\utils

24/07/2022  22:57    <DIR>          .
24/07/2022  22:57    <DIR>          ..
13/08/2022  18:03    <DIR>          __pycache__
24/07/2022  22:57               432 adversarial_creation.py
07/11/2021  18:37            11.457 project_utils.py
17/07/2022  15:27    <DIR>          tmp
13/08/2022  18:01            31.616 utils_guided_retraining.py
               3 File(s)         43.505 bytes
               4 Dir(s)  22.691.643.392 bytes free


In [6]:
# utils for project
#import utils_guided_retraining2 as utils
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [7]:
#cd '../notebooks/'

In [8]:
dataset = "gtsrb"

In [9]:
x_train,y_train = utils.get_data(dataset,"Train",verbose=True)
x_val,y_val = utils.get_data(dataset,"Val",verbose=True)
x_test,y_test = utils.get_data(dataset,"Test",verbose=True)

D:/data/x_train.npy
x_set len:  31366
D:/data/y_train.npy
y_set len:  31366
D:/data/x_val.npy
x_set len:  3919
D:/data/y_val.npy
y_set len:  3919
D:/data/x_test.npy
x_set len:  3923
D:/data/y_test.npy
y_set len:  3923


In [10]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",verbose=True)

D:/data_adversarial_july/gtsrb/train_and_adversary.npy
x_set len:  35287
D:/data_adversarial_july/gtsrb/train_and_adversary_labels.npy
y_set len:  35287


In [11]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

3921
3921


In [12]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [13]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

3921
3921


## ----

In [14]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02"

In [15]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
Model loaded correctly


In [16]:
print(len(x_adversary_training))

3921


In [17]:
print(x_adversary_training.shape)
len(x_adversary_training)//20

(3921, 48, 48, 1)


196

In [18]:
n_data_points = 190

In [19]:
import gc

## Loading LSA and DSA values 

In [20]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"


In [21]:
lsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/lsa_values.npy"


In [21]:
lsa_values = np.load(lsa_direction)[len(x_train):] 
print(len(lsa_values))


3921


In [25]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_adversary_training)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_adversary_training)


In [26]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  190
190
1 :
0  ->  380
380
2 :
0  ->  570
570
3 :
0  ->  760
760
4 :
0  ->  950
950
5 :
0  ->  1140
1140
6 :
0  ->  1330
1330
7 :
0  ->  1520
1520
8 :
0  ->  1710
1710
9 :
0  ->  1900
1900
10 :
0  ->  2090
2090
11 :
0  ->  2280
2280
12 :
0  ->  2470
2470
13 :
0  ->  2660
2660
14 :
0  ->  2850
2850
15 :
0  ->  3040
3040
16 :
0  ->  3230
3230
17 :
0  ->  3420
3420
18 :
0  ->  3610
3610
19 :
Last
0  ->  3921
3921


In [23]:
len(image_sets_lsa[-2])

3610

## Training guided by LSA

In [27]:
len(label_sets_lsa)

20

In [28]:
print(model_dir)

C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06


In [29]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [30]:
models_lsa[0].evaluate(x_test,y_test)

123/123 [==============================] - 41s 75ms/step - loss: 0.0595 - accuracy: 0.9859


[0.029302377253770828, 0.9928625822067261]

In [31]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n=n+1

Epoch 1/10
3/3 [==============================] - 26s 5s/step - loss: 5.0634 - accuracy: 0.0842 - val_loss: 1.0709 - val_accuracy: 0.8591
Epoch 2/10
3/3 [==============================] - 10s 5s/step - loss: 3.1304 - accuracy: 0.1368 - val_loss: 0.9860 - val_accuracy: 0.8625
Epoch 3/10
3/3 [==============================] - 10s 5s/step - loss: 2.8142 - accuracy: 0.2053 - val_loss: 0.2846 - val_accuracy: 0.9530
Epoch 4/10
3/3 [==============================] - 10s 5s/step - loss: 2.5054 - accuracy: 0.2684 - val_loss: 0.2401 - val_accuracy: 0.9533
Epoch 5/10
3/3 [==============================] - 10s 5s/step - loss: 2.3226 - accuracy: 0.2895 - val_loss: 0.1581 - val_accuracy: 0.9666
Epoch 6/10
3/3 [==============================] - 10s 5s/step - loss: 2.2003 - accuracy: 0.3526 - val_loss: 0.1099 - val_accuracy: 0.9793
Epoch 7/10
3/3 [==============================] - 10s 5s/step - loss: 1.9964 - accuracy: 0.3526 - val_loss: 0.1516 - val_accuracy: 0.9617
Epoch 8/10
3/3 [==================

In [32]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

Epoch 1/10
6/6 [==============================] - 18s 2s/step - loss: 5.1555 - accuracy: 0.0762 - val_loss: 0.5223 - val_accuracy: 0.9446
Epoch 2/10
6/6 [==============================] - 11s 2s/step - loss: 2.8275 - accuracy: 0.1997 - val_loss: 0.1522 - val_accuracy: 0.9732
Epoch 3/10
6/6 [==============================] - 11s 2s/step - loss: 2.4853 - accuracy: 0.3045 - val_loss: 0.1138 - val_accuracy: 0.9801
Epoch 4/10
6/6 [==============================] - 11s 2s/step - loss: 2.2249 - accuracy: 0.3695 - val_loss: 0.0727 - val_accuracy: 0.9855
Epoch 5/10
6/6 [==============================] - 11s 2s/step - loss: 1.8716 - accuracy: 0.4532 - val_loss: 0.0865 - val_accuracy: 0.9829
Epoch 6/10
6/6 [==============================] - 11s 2s/step - loss: 1.7309 - accuracy: 0.4950 - val_loss: 0.0963 - val_accuracy: 0.9788
Epoch 7/10
6/6 [==============================] - 11s 2s/step - loss: 1.5495 - accuracy: 0.5184 - val_loss: 0.1062 - val_accuracy: 0.9722
Epoch 8/10
6/6 [==================

In [33]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
9/9 [==============================] - 19s 2s/step - loss: 4.2448 - accuracy: 0.1190 - val_loss: 0.2246 - val_accuracy: 0.9543
Epoch 2/10
9/9 [==============================] - 13s 2s/step - loss: 2.5342 - accuracy: 0.2930 - val_loss: 0.1332 - val_accuracy: 0.9645
Epoch 3/10
9/9 [==============================] - 13s 2s/step - loss: 2.2093 - accuracy: 0.3871 - val_loss: 0.1069 - val_accuracy: 0.9775
Epoch 4/10
9/9 [==============================] - 13s 2s/step - loss: 1.8806 - accuracy: 0.4472 - val_loss: 0.0680 - val_accuracy: 0.9809
Epoch 5/10
9/9 [==============================] - 13s 2s/step - loss: 1.8627 - accuracy: 0.4925 - val_loss: 0.1235 - val_accuracy: 0.9684
Epoch 6/10
9/9 [==============================] - 13s 2s/step - loss: 1.5269 - accuracy: 0.5491 - val_loss: 0.1290 - val_accuracy: 0.9768
Epoch 7/10
9/9 [==============================] - 13s 2s/step - loss: 1.4119 - accuracy: 0.5676 - val_loss: 0.0939 - val_accuracy: 0.9775
Epoch 8/10
9/9 [================

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
12/12 [==============================] - 21s 1s/step - loss: 3.8552 - accuracy: 0.1724 - val_loss: 0.1491 - val_accuracy: 0.9691
Epoch 2/10
12/12 [==============================] - 14s 1s/step - loss: 2.2922 - accuracy: 0.3599 - val_loss: 0.1139 - val_accuracy: 0.9788
Epoch 3/10
12/12 [==============================] - 14s 1s/step - loss: 1.9166 - accuracy: 0.4694 - val_loss: 0.1296 - val_accuracy: 0.9684
Epoch 4/10
12/12 [==============================] - 14s 1s/step - loss: 1.8433 - accuracy: 0.4577 - val_loss: 0.1147 - val_accuracy: 0.9732
Epoch 5/10
12/12 [==============================] - 15s 1s/step - loss: 1.4544 - accuracy: 0.5771 - val_loss: 0.0898 - val_accuracy: 0.9755
Epoch 6/10
12/12 [==============================] - 14s 1s/step - loss: 1.3562 - accuracy: 0.5904 - val_loss: 0.0658 - val_accuracy: 0.9832
Epoch 7/10
12/12 [==============================] - 14s 1s/step - loss: 1.1732 - accuracy: 0.6329 - val_loss: 0.1090 - val_accuracy: 0.9696
Epoch 8/10
12/12 [

In [35]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
15/15 [==============================] - 22s 1s/step - loss: 3.7006 - accuracy: 0.1847 - val_loss: 0.2719 - val_accuracy: 0.9712
Epoch 2/10
15/15 [==============================] - 16s 1s/step - loss: 2.2243 - accuracy: 0.4002 - val_loss: 0.1233 - val_accuracy: 0.9786
Epoch 3/10
15/15 [==============================] - 16s 1s/step - loss: 1.8106 - accuracy: 0.5171 - val_loss: 0.0959 - val_accuracy: 0.9791
Epoch 4/10
15/15 [==============================] - 16s 1s/step - loss: 1.5397 - accuracy: 0.5587 - val_loss: 0.1065 - val_accuracy: 0.9732
Epoch 5/10
15/15 [==============================] - 16s 1s/step - loss: 1.3633 - accuracy: 0.6252 - val_loss: 0.0889 - val_accuracy: 0.9791
Epoch 6/10
15/15 [==============================] - 16s 1s/step - loss: 1.1799 - accuracy: 0.6378 - val_loss: 0.0806 - val_accuracy: 0.9804
Epoch 7/10
15/15 [==============================] - 16s 1s/step - loss: 1.0598 - accuracy: 0.6746 - val_loss: 0.2887 - val_accuracy: 0.9232
Epoch 8/10
15/15 [

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
18/18 [==============================] - 24s 1s/step - loss: 3.4402 - accuracy: 0.2367 - val_loss: 0.1233 - val_accuracy: 0.9788
Epoch 2/10
18/18 [==============================] - 18s 1s/step - loss: 1.8938 - accuracy: 0.4983 - val_loss: 0.1083 - val_accuracy: 0.9745
Epoch 3/10
18/18 [==============================] - 18s 1s/step - loss: 1.5133 - accuracy: 0.5612 - val_loss: 0.0775 - val_accuracy: 0.9804
Epoch 4/10
18/18 [==============================] - 18s 1s/step - loss: 1.2914 - accuracy: 0.6407 - val_loss: 0.0792 - val_accuracy: 0.9781
Epoch 5/10
18/18 [==============================] - 18s 1s/step - loss: 1.1351 - accuracy: 0.6761 - val_loss: 0.0746 - val_accuracy: 0.9821
Epoch 6/10
18/18 [==============================] - 18s 1s/step - loss: 1.0056 - accuracy: 0.7016 - val_loss: 0.1118 - val_accuracy: 0.9673
Epoch 7/10
18/18 [==============================] - 18s 1s/step - loss: 0.8780 - accuracy: 0.7339 - val_loss: 0.1147 - val_accuracy: 0.9701
Epoch 8/10
18/18 [

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
21/21 [==============================] - 25s 995ms/step - loss: 3.2076 - accuracy: 0.3119 - val_loss: 0.0929 - val_accuracy: 0.9862
Epoch 2/10
21/21 [==============================] - 19s 928ms/step - loss: 1.7846 - accuracy: 0.5224 - val_loss: 0.0723 - val_accuracy: 0.9814
Epoch 3/10
21/21 [==============================] - 19s 937ms/step - loss: 1.2823 - accuracy: 0.6376 - val_loss: 0.1146 - val_accuracy: 0.9709
Epoch 4/10
21/21 [==============================] - 19s 931ms/step - loss: 1.2477 - accuracy: 0.6576 - val_loss: 0.0764 - val_accuracy: 0.9791
Epoch 5/10
21/21 [==============================] - 19s 934ms/step - loss: 1.1412 - accuracy: 0.6668 - val_loss: 0.1016 - val_accuracy: 0.9752
Epoch 6/10
21/21 [==============================] - 19s 931ms/step - loss: 0.8314 - accuracy: 0.7629 - val_loss: 0.0825 - val_accuracy: 0.9781
Epoch 7/10
21/21 [==============================] - 19s 928ms/step - loss: 0.7239 - accuracy: 0.7791 - val_loss: 0.0933 - val_accuracy: 0.97

In [38]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
24/24 [==============================] - 27s 934ms/step - loss: 2.7136 - accuracy: 0.3608 - val_loss: 0.0490 - val_accuracy: 0.9865
Epoch 2/10
24/24 [==============================] - 21s 878ms/step - loss: 1.4996 - accuracy: 0.5798 - val_loss: 0.1974 - val_accuracy: 0.9617
Epoch 3/10
24/24 [==============================] - 21s 881ms/step - loss: 1.2761 - accuracy: 0.6263 - val_loss: 0.0950 - val_accuracy: 0.9796
Epoch 4/10
24/24 [==============================] - 21s 882ms/step - loss: 1.0138 - accuracy: 0.7129 - val_loss: 0.0676 - val_accuracy: 0.9826
Epoch 5/10
24/24 [==============================] - 21s 887ms/step - loss: 0.9455 - accuracy: 0.7202 - val_loss: 0.1495 - val_accuracy: 0.9666
Epoch 6/10
24/24 [==============================] - 21s 880ms/step - loss: 0.7690 - accuracy: 0.7795 - val_loss: 0.0888 - val_accuracy: 0.9758
Epoch 7/10
24/24 [==============================] - 21s 879ms/step - loss: 0.7662 - accuracy: 0.7716 - val_loss: 0.0901 - val_accuracy: 0.97

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
27/27 [==============================] - 29s 883ms/step - loss: 2.4858 - accuracy: 0.4176 - val_loss: 0.0576 - val_accuracy: 0.9880
Epoch 2/10
27/27 [==============================] - 22s 837ms/step - loss: 1.4175 - accuracy: 0.6190 - val_loss: 0.1138 - val_accuracy: 0.9740
Epoch 3/10
27/27 [==============================] - 22s 844ms/step - loss: 1.1113 - accuracy: 0.6931 - val_loss: 0.0923 - val_accuracy: 0.9806
Epoch 4/10
27/27 [==============================] - 22s 841ms/step - loss: 0.9864 - accuracy: 0.7085 - val_loss: 0.0659 - val_accuracy: 0.9847
Epoch 5/10
27/27 [==============================] - 23s 846ms/step - loss: 0.7959 - accuracy: 0.7770 - val_loss: 0.0980 - val_accuracy: 0.9732
Epoch 6/10
27/27 [==============================] - 23s 844ms/step - loss: 0.7468 - accuracy: 0.7757 - val_loss: 0.0889 - val_accuracy: 0.9786
Epoch 7/10
27/27 [==============================] - 22s 840ms/step - loss: 0.6845 - accuracy: 0.8004 - val_loss: 0.0610 - val_accuracy: 0.98

In [40]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
30/30 [==============================] - 29s 796ms/step - loss: 2.2495 - accuracy: 0.4695 - val_loss: 0.1411 - val_accuracy: 0.9681
Epoch 2/10
30/30 [==============================] - 23s 786ms/step - loss: 1.3290 - accuracy: 0.6479 - val_loss: 0.1028 - val_accuracy: 0.9707
Epoch 3/10
30/30 [==============================] - 24s 805ms/step - loss: 1.0491 - accuracy: 0.6982 - val_loss: 0.0610 - val_accuracy: 0.9849
Epoch 4/10
30/30 [==============================] - 24s 798ms/step - loss: 0.8451 - accuracy: 0.7602 - val_loss: 0.0862 - val_accuracy: 0.9763
Epoch 5/10
30/30 [==============================] - 24s 817ms/step - loss: 0.7358 - accuracy: 0.7986 - val_loss: 0.0677 - val_accuracy: 0.9809
Epoch 6/10
30/30 [==============================] - 24s 812ms/step - loss: 0.6044 - accuracy: 0.8183 - val_loss: 0.0704 - val_accuracy: 0.9793
Epoch 7/10
30/30 [==============================] - 24s 811ms/step - loss: 0.5423 - accuracy: 0.8407 - val_loss: 0.0814 - val_accuracy: 0.97

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
33/33 [==============================] - 31s 806ms/step - loss: 2.0632 - accuracy: 0.5040 - val_loss: 0.1830 - val_accuracy: 0.9781
Epoch 2/10
33/33 [==============================] - 26s 787ms/step - loss: 1.1547 - accuracy: 0.6747 - val_loss: 0.0908 - val_accuracy: 0.9765
Epoch 3/10
33/33 [==============================] - 26s 791ms/step - loss: 0.9379 - accuracy: 0.7316 - val_loss: 0.0409 - val_accuracy: 0.9888
Epoch 4/10
33/33 [==============================] - 26s 787ms/step - loss: 0.7867 - accuracy: 0.7757 - val_loss: 0.0512 - val_accuracy: 0.9847
Epoch 5/10
33/33 [==============================] - 26s 791ms/step - loss: 0.6290 - accuracy: 0.8112 - val_loss: 0.0528 - val_accuracy: 0.9857
Epoch 6/10
33/33 [==============================] - 26s 787ms/step - loss: 0.5830 - accuracy: 0.8329 - val_loss: 0.0636 - val_accuracy: 0.9834
Epoch 7/10
33/33 [==============================] - 26s 784ms/step - loss: 0.4857 - accuracy: 0.8558 - val_loss: 0.0775 - val_accuracy: 0.9

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
36/36 [==============================] - 35s 833ms/step - loss: 1.8528 - accuracy: 0.5554 - val_loss: 0.0650 - val_accuracy: 0.9883
Epoch 2/10
36/36 [==============================] - 27s 769ms/step - loss: 1.1053 - accuracy: 0.6922 - val_loss: 0.0407 - val_accuracy: 0.9893
Epoch 3/10
36/36 [==============================] - 27s 764ms/step - loss: 0.8517 - accuracy: 0.7469 - val_loss: 0.0808 - val_accuracy: 0.9791
Epoch 4/10
36/36 [==============================] - 27s 762ms/step - loss: 0.7435 - accuracy: 0.7821 - val_loss: 0.0597 - val_accuracy: 0.9847
Epoch 5/10
36/36 [==============================] - 27s 761ms/step - loss: 0.5772 - accuracy: 0.8188 - val_loss: 0.0649 - val_accuracy: 0.9826
Epoch 6/10
36/36 [==============================] - 27s 765ms/step - loss: 0.5292 - accuracy: 0.8453 - val_loss: 0.0819 - val_accuracy: 0.9791
Epoch 7/10
36/36 [==============================] - 27s 764ms/step - loss: 0.4713 - accuracy: 0.8545 - val_loss: 0.0928 - val_accuracy: 0.9

In [43]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
39/39 [==============================] - 35s 778ms/step - loss: 1.8142 - accuracy: 0.5864 - val_loss: 0.0965 - val_accuracy: 0.9783
Epoch 2/10
39/39 [==============================] - 29s 748ms/step - loss: 1.0405 - accuracy: 0.7078 - val_loss: 0.1022 - val_accuracy: 0.9849
Epoch 3/10
39/39 [==============================] - 29s 746ms/step - loss: 0.8144 - accuracy: 0.7666 - val_loss: 0.0493 - val_accuracy: 0.9878
Epoch 4/10
39/39 [==============================] - 29s 743ms/step - loss: 0.7212 - accuracy: 0.7804 - val_loss: 0.0608 - val_accuracy: 0.9847
Epoch 5/10
39/39 [==============================] - 29s 744ms/step - loss: 0.5963 - accuracy: 0.8282 - val_loss: 0.0610 - val_accuracy: 0.9852
Epoch 6/10
39/39 [==============================] - 29s 750ms/step - loss: 0.5475 - accuracy: 0.8396 - val_loss: 0.0532 - val_accuracy: 0.9865
Epoch 7/10
39/39 [==============================] - 29s 746ms/step - loss: 0.4998 - accuracy: 0.8701 - val_loss: 0.0537 - val_accuracy: 0.9

In [44]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
42/42 [==============================] - 37s 760ms/step - loss: 1.7501 - accuracy: 0.5900 - val_loss: 0.0534 - val_accuracy: 0.9898
Epoch 2/10
42/42 [==============================] - 31s 731ms/step - loss: 0.8867 - accuracy: 0.7511 - val_loss: 0.0550 - val_accuracy: 0.9875
Epoch 3/10
42/42 [==============================] - 30s 730ms/step - loss: 0.7346 - accuracy: 0.7834 - val_loss: 0.0548 - val_accuracy: 0.9855
Epoch 4/10
42/42 [==============================] - 31s 735ms/step - loss: 0.6615 - accuracy: 0.8068 - val_loss: 0.0577 - val_accuracy: 0.9862
Epoch 5/10
42/42 [==============================] - 30s 728ms/step - loss: 0.5542 - accuracy: 0.8426 - val_loss: 0.0549 - val_accuracy: 0.9847
Epoch 6/10
42/42 [==============================] - 30s 729ms/step - loss: 0.4902 - accuracy: 0.8561 - val_loss: 0.0638 - val_accuracy: 0.9816
Epoch 7/10
42/42 [==============================] - 30s 729ms/step - loss: 0.4025 - accuracy: 0.8801 - val_loss: 0.0904 - val_accuracy: 0.9

In [45]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
45/45 [==============================] - 39s 747ms/step - loss: 1.6546 - accuracy: 0.6134 - val_loss: 0.0507 - val_accuracy: 0.9862
Epoch 2/10
45/45 [==============================] - 33s 728ms/step - loss: 0.8987 - accuracy: 0.7518 - val_loss: 0.1060 - val_accuracy: 0.9760
Epoch 3/10
45/45 [==============================] - 32s 717ms/step - loss: 0.7673 - accuracy: 0.7895 - val_loss: 0.0407 - val_accuracy: 0.9898
Epoch 4/10
45/45 [==============================] - 32s 717ms/step - loss: 0.5812 - accuracy: 0.8340 - val_loss: 0.0867 - val_accuracy: 0.9791
Epoch 5/10
45/45 [==============================] - 32s 709ms/step - loss: 0.5639 - accuracy: 0.8383 - val_loss: 0.0626 - val_accuracy: 0.9832
Epoch 6/10
45/45 [==============================] - 32s 716ms/step - loss: 0.4724 - accuracy: 0.8647 - val_loss: 0.0688 - val_accuracy: 0.9829
Epoch 7/10
45/45 [==============================] - 32s 715ms/step - loss: 0.3554 - accuracy: 0.8841 - val_loss: 0.0642 - val_accuracy: 0.9

In [46]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
48/48 [==============================] - 40s 728ms/step - loss: 1.4962 - accuracy: 0.6477 - val_loss: 0.1187 - val_accuracy: 0.9694
Epoch 2/10
48/48 [==============================] - 33s 696ms/step - loss: 0.8938 - accuracy: 0.7549 - val_loss: 0.3065 - val_accuracy: 0.9153
Epoch 3/10
48/48 [==============================] - 34s 705ms/step - loss: 0.6794 - accuracy: 0.8134 - val_loss: 0.0598 - val_accuracy: 0.9860
Epoch 4/10
48/48 [==============================] - 33s 695ms/step - loss: 0.5877 - accuracy: 0.8313 - val_loss: 0.0666 - val_accuracy: 0.9786
Epoch 5/10
48/48 [==============================] - 33s 703ms/step - loss: 0.4942 - accuracy: 0.8527 - val_loss: 0.0564 - val_accuracy: 0.9844
Epoch 6/10
48/48 [==============================] - 34s 706ms/step - loss: 0.4577 - accuracy: 0.8644 - val_loss: 0.0577 - val_accuracy: 0.9844
Epoch 7/10
48/48 [==============================] - 34s 708ms/step - loss: 0.4061 - accuracy: 0.8820 - val_loss: 0.0541 - val_accuracy: 0.9

In [47]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
51/51 [==============================] - 42s 721ms/step - loss: 1.4425 - accuracy: 0.6718 - val_loss: 0.0658 - val_accuracy: 0.9834
Epoch 2/10
51/51 [==============================] - 35s 694ms/step - loss: 0.7902 - accuracy: 0.7766 - val_loss: 0.0740 - val_accuracy: 0.9829
Epoch 3/10
51/51 [==============================] - 35s 698ms/step - loss: 0.5839 - accuracy: 0.8343 - val_loss: 0.0974 - val_accuracy: 0.9758
Epoch 4/10
51/51 [==============================] - 35s 697ms/step - loss: 0.5451 - accuracy: 0.8395 - val_loss: 0.0439 - val_accuracy: 0.9885
Epoch 5/10
51/51 [==============================] - 35s 693ms/step - loss: 0.4384 - accuracy: 0.8675 - val_loss: 0.0771 - val_accuracy: 0.9821
Epoch 6/10
51/51 [==============================] - 35s 697ms/step - loss: 0.4574 - accuracy: 0.8707 - val_loss: 0.0643 - val_accuracy: 0.9860
Epoch 7/10
51/51 [==============================] - 35s 694ms/step - loss: 0.3392 - accuracy: 0.8935 - val_loss: 0.0589 - val_accuracy: 0.9

In [48]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
54/54 [==============================] - 43s 710ms/step - loss: 1.3102 - accuracy: 0.6954 - val_loss: 0.0573 - val_accuracy: 0.9870
Epoch 2/10
54/54 [==============================] - 37s 684ms/step - loss: 0.7591 - accuracy: 0.7887 - val_loss: 0.0614 - val_accuracy: 0.9867
Epoch 3/10
54/54 [==============================] - 37s 689ms/step - loss: 0.6069 - accuracy: 0.8284 - val_loss: 0.0431 - val_accuracy: 0.9890
Epoch 4/10
54/54 [==============================] - 37s 684ms/step - loss: 0.5549 - accuracy: 0.8408 - val_loss: 0.0682 - val_accuracy: 0.9793
Epoch 5/10
54/54 [==============================] - 37s 685ms/step - loss: 0.4325 - accuracy: 0.8778 - val_loss: 0.0591 - val_accuracy: 0.9855
Epoch 6/10
54/54 [==============================] - 37s 689ms/step - loss: 0.3892 - accuracy: 0.8897 - val_loss: 0.0398 - val_accuracy: 0.9916
Epoch 7/10
54/54 [==============================] - 37s 688ms/step - loss: 0.3573 - accuracy: 0.9003 - val_loss: 0.0528 - val_accuracy: 0.9

In [49]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
57/57 [==============================] - 45s 700ms/step - loss: 1.3098 - accuracy: 0.7023 - val_loss: 0.0754 - val_accuracy: 0.9875
Epoch 2/10
57/57 [==============================] - 39s 679ms/step - loss: 0.6558 - accuracy: 0.8212 - val_loss: 0.0988 - val_accuracy: 0.9758
Epoch 3/10
57/57 [==============================] - 36s 634ms/step - loss: 0.5777 - accuracy: 0.8363 - val_loss: 0.0485 - val_accuracy: 0.9893
Epoch 4/10
57/57 [==============================] - 38s 675ms/step - loss: 0.5015 - accuracy: 0.8616 - val_loss: 0.1423 - val_accuracy: 0.9712
Epoch 5/10
57/57 [==============================] - 38s 675ms/step - loss: 0.4336 - accuracy: 0.8706 - val_loss: 0.0816 - val_accuracy: 0.9747
Epoch 6/10
57/57 [==============================] - 38s 674ms/step - loss: 0.3785 - accuracy: 0.8913 - val_loss: 0.0979 - val_accuracy: 0.9737
Epoch 7/10
57/57 [==============================] - 38s 675ms/step - loss: 0.3670 - accuracy: 0.8957 - val_loss: 0.0872 - val_accuracy: 0.9

In [50]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
62/62 [==============================] - 48s 696ms/step - loss: 1.1301 - accuracy: 0.7375 - val_loss: 0.0365 - val_accuracy: 0.9923
Epoch 2/10
62/62 [==============================] - 42s 676ms/step - loss: 0.6903 - accuracy: 0.8102 - val_loss: 0.0579 - val_accuracy: 0.9878
Epoch 3/10
62/62 [==============================] - 42s 677ms/step - loss: 0.5567 - accuracy: 0.8476 - val_loss: 0.0556 - val_accuracy: 0.9847
Epoch 4/10
62/62 [==============================] - 42s 680ms/step - loss: 0.4650 - accuracy: 0.8720 - val_loss: 0.0562 - val_accuracy: 0.9857
Epoch 5/10
62/62 [==============================] - 42s 679ms/step - loss: 0.4001 - accuracy: 0.8849 - val_loss: 0.0596 - val_accuracy: 0.9852
Epoch 6/10
62/62 [==============================] - 42s 684ms/step - loss: 0.3600 - accuracy: 0.8965 - val_loss: 0.0525 - val_accuracy: 0.9852
Epoch 7/10
62/62 [==============================] - 42s 682ms/step - loss: 0.3034 - accuracy: 0.9113 - val_loss: 0.2340 - val_accuracy: 0.9

In [51]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C3/gtsrb_model_c3_sep_lsa_e1"
new_model_lsa_dir  = "D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2"
#new_model_lsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_lsa_e2_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [52]:
import gc

In [53]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [54]:
gc.collect()

589658

## Training guided by DSA

In [62]:
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [63]:
#dsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/dsa_values.npy"

In [64]:
dsa_values = np.load(dsa_direction)[len(x_train):] 
print(len(dsa_values))

3921


In [65]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_adversary_training)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_adversary_training)

In [66]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  190
190
1 :
0  ->  380
380
2 :
0  ->  570
570
3 :
0  ->  760
760
4 :
0  ->  950
950
5 :
0  ->  1140
1140
6 :
0  ->  1330
1330
7 :
0  ->  1520
1520
8 :
0  ->  1710
1710
9 :
0  ->  1900
1900
10 :
0  ->  2090
2090
11 :
0  ->  2280
2280
12 :
0  ->  2470
2470
13 :
0  ->  2660
2660
14 :
0  ->  2850
2850
15 :
0  ->  3040
3040
16 :
0  ->  3230
3230
17 :
0  ->  3420
3420
18 :
0  ->  3610
3610
19 :
Last
0  ->  3921
3921


In [67]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [68]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

0
Epoch 1/10
3/3 [==============================] - 16s 5s/step - loss: 5.3541 - accuracy: 0.0531 - val_loss: 0.9613 - val_accuracy: 0.8369
Epoch 2/10
3/3 [==============================] - 11s 5s/step - loss: 3.5157 - accuracy: 0.1165 - val_loss: 0.5402 - val_accuracy: 0.9398
Epoch 3/10
3/3 [==============================] - 10s 5s/step - loss: 3.1249 - accuracy: 0.1754 - val_loss: 0.1855 - val_accuracy: 0.9676
Epoch 4/10
3/3 [==============================] - 6s 3s/step - loss: 2.8232 - accuracy: 0.2597 - val_loss: 0.1578 - val_accuracy: 0.9694
Epoch 5/10
3/3 [==============================] - 7s 3s/step - loss: 2.5141 - accuracy: 0.3127 - val_loss: 0.2006 - val_accuracy: 0.9566
Epoch 6/10
3/3 [==============================] - 8s 4s/step - loss: 2.2739 - accuracy: 0.3632 - val_loss: 0.1390 - val_accuracy: 0.9732
Epoch 7/10
3/3 [==============================] - 8s 4s/step - loss: 2.1445 - accuracy: 0.3744 - val_loss: 0.1895 - val_accuracy: 0.9604
Epoch 8/10
3/3 [====================

In [69]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

1
Epoch 1/10
6/6 [==============================] - 14s 2s/step - loss: 4.6899 - accuracy: 0.0802 - val_loss: 0.3548 - val_accuracy: 0.9479
Epoch 2/10
6/6 [==============================] - 9s 2s/step - loss: 2.9289 - accuracy: 0.2137 - val_loss: 0.5527 - val_accuracy: 0.8908
Epoch 3/10
6/6 [==============================] - 9s 2s/step - loss: 2.6561 - accuracy: 0.3160 - val_loss: 0.2270 - val_accuracy: 0.9508
Epoch 4/10
6/6 [==============================] - 8s 2s/step - loss: 2.1294 - accuracy: 0.3961 - val_loss: 0.6538 - val_accuracy: 0.8599
Epoch 5/10
6/6 [==============================] - 8s 2s/step - loss: 2.0321 - accuracy: 0.4347 - val_loss: 0.3429 - val_accuracy: 0.9265
Epoch 6/10
6/6 [==============================] - 8s 2s/step - loss: 1.8068 - accuracy: 0.4834 - val_loss: 0.2433 - val_accuracy: 0.9421
Epoch 7/10
6/6 [==============================] - 8s 2s/step - loss: 1.6363 - accuracy: 0.5204 - val_loss: 0.3339 - val_accuracy: 0.9189
Epoch 8/10
6/6 [======================

In [70]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
9/9 [==============================] - 14s 1s/step - loss: 4.3875 - accuracy: 0.0952 - val_loss: 0.1517 - val_accuracy: 0.9786
Epoch 2/10
9/9 [==============================] - 11s 1s/step - loss: 2.7513 - accuracy: 0.2636 - val_loss: 0.1434 - val_accuracy: 0.9793
Epoch 3/10
9/9 [==============================] - 11s 1s/step - loss: 2.2559 - accuracy: 0.3507 - val_loss: 0.2523 - val_accuracy: 0.9472
Epoch 4/10
9/9 [==============================] - 11s 1s/step - loss: 1.9522 - accuracy: 0.4626 - val_loss: 0.2459 - val_accuracy: 0.9530
Epoch 5/10
9/9 [==============================] - 11s 1s/step - loss: 1.7641 - accuracy: 0.5038 - val_loss: 0.2136 - val_accuracy: 0.9444
Epoch 6/10
9/9 [==============================] - 11s 1s/step - loss: 1.4884 - accuracy: 0.5843 - val_loss: 0.3110 - val_accuracy: 0.9183
Epoch 7/10
9/9 [==============================] - 11s 1s/step - loss: 1.4041 - accuracy: 0.5767 - val_loss: 0.3521 - val_accuracy: 0.9178
Epoch 8/10
9/9 [================

In [71]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
12/12 [==============================] - 17s 1s/step - loss: 4.0456 - accuracy: 0.1119 - val_loss: 0.3148 - val_accuracy: 0.9257
Epoch 2/10
12/12 [==============================] - 13s 1s/step - loss: 2.6073 - accuracy: 0.3255 - val_loss: 0.2339 - val_accuracy: 0.9367
Epoch 3/10
12/12 [==============================] - 13s 1s/step - loss: 2.1317 - accuracy: 0.4076 - val_loss: 0.4591 - val_accuracy: 0.9173
Epoch 4/10
12/12 [==============================] - 13s 1s/step - loss: 1.9416 - accuracy: 0.4592 - val_loss: 0.2884 - val_accuracy: 0.9382
Epoch 5/10
12/12 [==============================] - 13s 1s/step - loss: 1.5139 - accuracy: 0.5680 - val_loss: 0.3014 - val_accuracy: 0.9257
Epoch 6/10
12/12 [==============================] - 13s 1s/step - loss: 1.4007 - accuracy: 0.5670 - val_loss: 0.4829 - val_accuracy: 0.8870
Epoch 7/10
12/12 [==============================] - 13s 1s/step - loss: 1.2185 - accuracy: 0.6278 - val_loss: 0.4248 - val_accuracy: 0.8977
Epoch 8/10
12/12 [

In [72]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
15/15 [==============================] - 17s 1s/step - loss: 3.7191 - accuracy: 0.1456 - val_loss: 0.1249 - val_accuracy: 0.9684
Epoch 2/10
15/15 [==============================] - 15s 1s/step - loss: 2.2753 - accuracy: 0.3891 - val_loss: 0.1595 - val_accuracy: 0.9569
Epoch 3/10
15/15 [==============================] - 15s 1s/step - loss: 1.8466 - accuracy: 0.4875 - val_loss: 0.2892 - val_accuracy: 0.9324
Epoch 4/10
15/15 [==============================] - 15s 1s/step - loss: 1.7319 - accuracy: 0.5406 - val_loss: 0.2477 - val_accuracy: 0.9405
Epoch 5/10
15/15 [==============================] - 15s 1s/step - loss: 1.3495 - accuracy: 0.6228 - val_loss: 0.1931 - val_accuracy: 0.9610
Epoch 6/10
15/15 [==============================] - 15s 1s/step - loss: 1.2549 - accuracy: 0.6148 - val_loss: 0.2441 - val_accuracy: 0.9342
Epoch 7/10
15/15 [==============================] - 15s 1s/step - loss: 1.0645 - accuracy: 0.6979 - val_loss: 0.1500 - val_accuracy: 0.9594
Epoch 8/10
15/15 [

In [73]:
n=5
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
18/18 [==============================] - 19s 939ms/step - loss: 3.6106 - accuracy: 0.1707 - val_loss: 0.0829 - val_accuracy: 0.9832
Epoch 2/10
18/18 [==============================] - 17s 987ms/step - loss: 2.0741 - accuracy: 0.4520 - val_loss: 0.1059 - val_accuracy: 0.9773
Epoch 3/10
18/18 [==============================] - 17s 945ms/step - loss: 1.6579 - accuracy: 0.5325 - val_loss: 0.2224 - val_accuracy: 0.9490
Epoch 4/10
18/18 [==============================] - 16s 934ms/step - loss: 1.3074 - accuracy: 0.6306 - val_loss: 0.2083 - val_accuracy: 0.9405
Epoch 5/10
18/18 [==============================] - 16s 931ms/step - loss: 1.2583 - accuracy: 0.6327 - val_loss: 0.1575 - val_accuracy: 0.9592
Epoch 6/10
18/18 [==============================] - 16s 933ms/step - loss: 1.0659 - accuracy: 0.6870 - val_loss: 0.1812 - val_accuracy: 0.9456
Epoch 7/10
18/18 [==============================] - 17s 940ms/step - loss: 1.0491 - accuracy: 0.6922 - val_loss: 0.1796 - val_accuracy: 0.95

In [74]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
21/21 [==============================] - 23s 988ms/step - loss: 3.6093 - accuracy: 0.2050 - val_loss: 0.1363 - val_accuracy: 0.9742
Epoch 2/10
21/21 [==============================] - 19s 924ms/step - loss: 1.8214 - accuracy: 0.4721 - val_loss: 0.2498 - val_accuracy: 0.9643
Epoch 3/10
21/21 [==============================] - 19s 932ms/step - loss: 1.6184 - accuracy: 0.5605 - val_loss: 0.0853 - val_accuracy: 0.9765
Epoch 4/10
21/21 [==============================] - 19s 921ms/step - loss: 1.2040 - accuracy: 0.6383 - val_loss: 0.0838 - val_accuracy: 0.9760
Epoch 5/10
21/21 [==============================] - 19s 904ms/step - loss: 1.1945 - accuracy: 0.6370 - val_loss: 0.0996 - val_accuracy: 0.9775
Epoch 6/10
21/21 [==============================] - 19s 905ms/step - loss: 1.0625 - accuracy: 0.6849 - val_loss: 0.0750 - val_accuracy: 0.9829
Epoch 7/10
21/21 [==============================] - 19s 913ms/step - loss: 0.8986 - accuracy: 0.7289 - val_loss: 0.0971 - val_accuracy: 0.97

In [75]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
24/24 [==============================] - 24s 908ms/step - loss: 3.1220 - accuracy: 0.2769 - val_loss: 0.1443 - val_accuracy: 0.9696
Epoch 2/10
24/24 [==============================] - 21s 874ms/step - loss: 1.7567 - accuracy: 0.4993 - val_loss: 0.1074 - val_accuracy: 0.9740
Epoch 3/10
24/24 [==============================] - 21s 878ms/step - loss: 1.3767 - accuracy: 0.6147 - val_loss: 0.0714 - val_accuracy: 0.9834
Epoch 4/10
24/24 [==============================] - 21s 870ms/step - loss: 1.1233 - accuracy: 0.6754 - val_loss: 0.0510 - val_accuracy: 0.9872
Epoch 5/10
24/24 [==============================] - 21s 870ms/step - loss: 0.9820 - accuracy: 0.7243 - val_loss: 0.0593 - val_accuracy: 0.9842
Epoch 6/10
24/24 [==============================] - 21s 875ms/step - loss: 0.8306 - accuracy: 0.7429 - val_loss: 0.0739 - val_accuracy: 0.9811
Epoch 7/10
24/24 [==============================] - 21s 875ms/step - loss: 0.7822 - accuracy: 0.7823 - val_loss: 0.0953 - val_accuracy: 0.97

In [76]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
27/27 [==============================] - 24s 808ms/step - loss: 2.6796 - accuracy: 0.3791 - val_loss: 0.1106 - val_accuracy: 0.9750
Epoch 2/10
27/27 [==============================] - 22s 817ms/step - loss: 1.4495 - accuracy: 0.5969 - val_loss: 0.0680 - val_accuracy: 0.9809
Epoch 3/10
27/27 [==============================] - 22s 809ms/step - loss: 1.2078 - accuracy: 0.6580 - val_loss: 0.0663 - val_accuracy: 0.9855
Epoch 4/10
27/27 [==============================] - 22s 816ms/step - loss: 1.0396 - accuracy: 0.7051 - val_loss: 0.0713 - val_accuracy: 0.9811
Epoch 5/10
27/27 [==============================] - 22s 812ms/step - loss: 0.8937 - accuracy: 0.7347 - val_loss: 0.1198 - val_accuracy: 0.9737
Epoch 6/10
27/27 [==============================] - 22s 815ms/step - loss: 0.8122 - accuracy: 0.7728 - val_loss: 0.0711 - val_accuracy: 0.9832
Epoch 7/10
27/27 [==============================] - 22s 816ms/step - loss: 0.6418 - accuracy: 0.8157 - val_loss: 0.0787 - val_accuracy: 0.97

In [77]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
30/30 [==============================] - 25s 777ms/step - loss: 2.7048 - accuracy: 0.3975 - val_loss: 0.0389 - val_accuracy: 0.9908
Epoch 2/10
30/30 [==============================] - 23s 771ms/step - loss: 1.3651 - accuracy: 0.6160 - val_loss: 0.0568 - val_accuracy: 0.9826
Epoch 3/10
30/30 [==============================] - 23s 770ms/step - loss: 1.1822 - accuracy: 0.6681 - val_loss: 0.0489 - val_accuracy: 0.9872
Epoch 4/10
30/30 [==============================] - 23s 770ms/step - loss: 0.9562 - accuracy: 0.7285 - val_loss: 0.0400 - val_accuracy: 0.9898
Epoch 5/10
30/30 [==============================] - 23s 776ms/step - loss: 0.8220 - accuracy: 0.7615 - val_loss: 0.0618 - val_accuracy: 0.9837
Epoch 6/10
30/30 [==============================] - 23s 777ms/step - loss: 0.6711 - accuracy: 0.8006 - val_loss: 0.0436 - val_accuracy: 0.9888
Epoch 7/10
30/30 [==============================] - 25s 841ms/step - loss: 0.6563 - accuracy: 0.8095 - val_loss: 0.0540 - val_accuracy: 0.98

In [78]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
33/33 [==============================] - 31s 839ms/step - loss: 2.4187 - accuracy: 0.4581 - val_loss: 0.2031 - val_accuracy: 0.9658
Epoch 2/10
33/33 [==============================] - 27s 819ms/step - loss: 1.2601 - accuracy: 0.6443 - val_loss: 0.0460 - val_accuracy: 0.9878
Epoch 3/10
33/33 [==============================] - 27s 818ms/step - loss: 1.0535 - accuracy: 0.6979 - val_loss: 0.0650 - val_accuracy: 0.9844
Epoch 4/10
33/33 [==============================] - 27s 822ms/step - loss: 0.8158 - accuracy: 0.7618 - val_loss: 0.0532 - val_accuracy: 0.9878
Epoch 5/10
33/33 [==============================] - 27s 819ms/step - loss: 0.7027 - accuracy: 0.7995 - val_loss: 0.0673 - val_accuracy: 0.9849
Epoch 6/10
33/33 [==============================] - 27s 824ms/step - loss: 0.6586 - accuracy: 0.8063 - val_loss: 0.0749 - val_accuracy: 0.9804
Epoch 7/10
33/33 [==============================] - 27s 820ms/step - loss: 0.5847 - accuracy: 0.8283 - val_loss: 0.0485 - val_accuracy: 0.9

In [79]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
36/36 [==============================] - 33s 816ms/step - loss: 2.1603 - accuracy: 0.5139 - val_loss: 0.0558 - val_accuracy: 0.9872
Epoch 2/10
36/36 [==============================] - 28s 795ms/step - loss: 1.2239 - accuracy: 0.6650 - val_loss: 0.0812 - val_accuracy: 0.9788
Epoch 3/10
36/36 [==============================] - 28s 791ms/step - loss: 0.9220 - accuracy: 0.7423 - val_loss: 0.0727 - val_accuracy: 0.9826
Epoch 4/10
36/36 [==============================] - 28s 794ms/step - loss: 0.7954 - accuracy: 0.7618 - val_loss: 0.0427 - val_accuracy: 0.9883
Epoch 5/10
36/36 [==============================] - 28s 795ms/step - loss: 0.7036 - accuracy: 0.7956 - val_loss: 0.0685 - val_accuracy: 0.9826
Epoch 6/10
36/36 [==============================] - 29s 822ms/step - loss: 0.5581 - accuracy: 0.8304 - val_loss: 0.0570 - val_accuracy: 0.9847
Epoch 7/10
36/36 [==============================] - 29s 812ms/step - loss: 0.5376 - accuracy: 0.8387 - val_loss: 0.0612 - val_accuracy: 0.9

In [80]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
39/39 [==============================] - 34s 799ms/step - loss: 1.9132 - accuracy: 0.5537 - val_loss: 0.0983 - val_accuracy: 0.9862
Epoch 2/10
39/39 [==============================] - 31s 798ms/step - loss: 1.0645 - accuracy: 0.7048 - val_loss: 0.0785 - val_accuracy: 0.9793
Epoch 3/10
39/39 [==============================] - 31s 793ms/step - loss: 0.8484 - accuracy: 0.7499 - val_loss: 0.0414 - val_accuracy: 0.9878
Epoch 4/10
39/39 [==============================] - 31s 789ms/step - loss: 0.7302 - accuracy: 0.7963 - val_loss: 0.0573 - val_accuracy: 0.9855
Epoch 5/10
39/39 [==============================] - 31s 788ms/step - loss: 0.6352 - accuracy: 0.8074 - val_loss: 0.0462 - val_accuracy: 0.9888
Epoch 6/10
39/39 [==============================] - 31s 790ms/step - loss: 0.5530 - accuracy: 0.8376 - val_loss: 0.0489 - val_accuracy: 0.9867
Epoch 7/10
39/39 [==============================] - 31s 790ms/step - loss: 0.4803 - accuracy: 0.8508 - val_loss: 0.0428 - val_accuracy: 0.9

In [81]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
42/42 [==============================] - 37s 805ms/step - loss: 1.7901 - accuracy: 0.5911 - val_loss: 0.0471 - val_accuracy: 0.9903
Epoch 2/10
42/42 [==============================] - 33s 795ms/step - loss: 1.0068 - accuracy: 0.7136 - val_loss: 0.0534 - val_accuracy: 0.9860
Epoch 3/10
42/42 [==============================] - 33s 789ms/step - loss: 0.8007 - accuracy: 0.7621 - val_loss: 0.0295 - val_accuracy: 0.9916
Epoch 4/10
42/42 [==============================] - 33s 793ms/step - loss: 0.6883 - accuracy: 0.8040 - val_loss: 0.0559 - val_accuracy: 0.9842
Epoch 5/10
42/42 [==============================] - 33s 793ms/step - loss: 0.5775 - accuracy: 0.8299 - val_loss: 0.0632 - val_accuracy: 0.9826
Epoch 6/10
42/42 [==============================] - 33s 799ms/step - loss: 0.5173 - accuracy: 0.8495 - val_loss: 0.0578 - val_accuracy: 0.9852
Epoch 7/10
42/42 [==============================] - 33s 796ms/step - loss: 0.4191 - accuracy: 0.8800 - val_loss: 0.0508 - val_accuracy: 0.9

In [82]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
45/45 [==============================] - 39s 795ms/step - loss: 1.6280 - accuracy: 0.6312 - val_loss: 0.0404 - val_accuracy: 0.9895
Epoch 2/10
45/45 [==============================] - 35s 791ms/step - loss: 0.9334 - accuracy: 0.7406 - val_loss: 0.0477 - val_accuracy: 0.9883
Epoch 3/10
45/45 [==============================] - 36s 794ms/step - loss: 0.6549 - accuracy: 0.8080 - val_loss: 0.0568 - val_accuracy: 0.9860
Epoch 4/10
45/45 [==============================] - 35s 788ms/step - loss: 0.6464 - accuracy: 0.8130 - val_loss: 0.0401 - val_accuracy: 0.9913
Epoch 5/10
45/45 [==============================] - 36s 798ms/step - loss: 0.4832 - accuracy: 0.8566 - val_loss: 0.1629 - val_accuracy: 0.9556
Epoch 6/10
45/45 [==============================] - 35s 788ms/step - loss: 0.5533 - accuracy: 0.8438 - val_loss: 0.0572 - val_accuracy: 0.9880
Epoch 7/10
45/45 [==============================] - 35s 788ms/step - loss: 0.4631 - accuracy: 0.8604 - val_loss: 0.0529 - val_accuracy: 0.9

In [83]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
48/48 [==============================] - 41s 788ms/step - loss: 1.4544 - accuracy: 0.6502 - val_loss: 0.0407 - val_accuracy: 0.9918
Epoch 2/10
48/48 [==============================] - 38s 797ms/step - loss: 0.8589 - accuracy: 0.7611 - val_loss: 0.0698 - val_accuracy: 0.9875
Epoch 3/10
48/48 [==============================] - 38s 794ms/step - loss: 0.6941 - accuracy: 0.7918 - val_loss: 0.0663 - val_accuracy: 0.9824
Epoch 4/10
48/48 [==============================] - 38s 791ms/step - loss: 0.5831 - accuracy: 0.8284 - val_loss: 0.0818 - val_accuracy: 0.9747
Epoch 5/10
48/48 [==============================] - 38s 788ms/step - loss: 0.4903 - accuracy: 0.8633 - val_loss: 0.0515 - val_accuracy: 0.9867
Epoch 6/10
48/48 [==============================] - 38s 794ms/step - loss: 0.4313 - accuracy: 0.8731 - val_loss: 0.0390 - val_accuracy: 0.9885
Epoch 7/10
48/48 [==============================] - 38s 792ms/step - loss: 0.4177 - accuracy: 0.8792 - val_loss: 0.0407 - val_accuracy: 0.9

In [84]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
51/51 [==============================] - 43s 785ms/step - loss: 1.4656 - accuracy: 0.6663 - val_loss: 0.0547 - val_accuracy: 0.9878
Epoch 2/10
51/51 [==============================] - 39s 776ms/step - loss: 0.8428 - accuracy: 0.7717 - val_loss: 0.0376 - val_accuracy: 0.9913
Epoch 3/10
51/51 [==============================] - 39s 760ms/step - loss: 0.6558 - accuracy: 0.8270 - val_loss: 0.0352 - val_accuracy: 0.9929
Epoch 4/10
51/51 [==============================] - 39s 764ms/step - loss: 0.5478 - accuracy: 0.8355 - val_loss: 0.0323 - val_accuracy: 0.9908
Epoch 5/10
51/51 [==============================] - 39s 765ms/step - loss: 0.5144 - accuracy: 0.8527 - val_loss: 0.0347 - val_accuracy: 0.9895
Epoch 6/10
51/51 [==============================] - 39s 765ms/step - loss: 0.4092 - accuracy: 0.8790 - val_loss: 0.0394 - val_accuracy: 0.9900
Epoch 7/10
51/51 [==============================] - 39s 764ms/step - loss: 0.3953 - accuracy: 0.8818 - val_loss: 0.0531 - val_accuracy: 0.9

In [85]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
54/54 [==============================] - 46s 793ms/step - loss: 1.4143 - accuracy: 0.6662 - val_loss: 0.0419 - val_accuracy: 0.9898
Epoch 2/10
54/54 [==============================] - 42s 786ms/step - loss: 0.8061 - accuracy: 0.7795 - val_loss: 0.0394 - val_accuracy: 0.9913
Epoch 3/10
54/54 [==============================] - 42s 782ms/step - loss: 0.6337 - accuracy: 0.8260 - val_loss: 0.0458 - val_accuracy: 0.9878
Epoch 4/10
54/54 [==============================] - 42s 789ms/step - loss: 0.5284 - accuracy: 0.8463 - val_loss: 0.0444 - val_accuracy: 0.9903
Epoch 5/10
54/54 [==============================] - 43s 790ms/step - loss: 0.4437 - accuracy: 0.8745 - val_loss: 0.0577 - val_accuracy: 0.9878
Epoch 6/10
54/54 [==============================] - 42s 789ms/step - loss: 0.4369 - accuracy: 0.8739 - val_loss: 0.0410 - val_accuracy: 0.9906
Epoch 7/10
54/54 [==============================] - 42s 787ms/step - loss: 0.3634 - accuracy: 0.8941 - val_loss: 0.0385 - val_accuracy: 0.9

In [86]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
57/57 [==============================] - 49s 790ms/step - loss: 1.4682 - accuracy: 0.6841 - val_loss: 0.0385 - val_accuracy: 0.9903
Epoch 2/10
57/57 [==============================] - 45s 793ms/step - loss: 0.7411 - accuracy: 0.8069 - val_loss: 0.0602 - val_accuracy: 0.9857
Epoch 3/10
57/57 [==============================] - 45s 799ms/step - loss: 0.5991 - accuracy: 0.8259 - val_loss: 0.0410 - val_accuracy: 0.9900
Epoch 4/10
57/57 [==============================] - 77s 1s/step - loss: 0.4993 - accuracy: 0.8526 - val_loss: 0.0636 - val_accuracy: 0.9860
Epoch 5/10
57/57 [==============================] - 80s 1s/step - loss: 0.4698 - accuracy: 0.8738 - val_loss: 0.0420 - val_accuracy: 0.9870
Epoch 6/10
57/57 [==============================] - 80s 1s/step - loss: 0.3927 - accuracy: 0.8811 - val_loss: 0.0396 - val_accuracy: 0.9900
Epoch 7/10
57/57 [==============================] - 79s 1s/step - loss: 0.3593 - accuracy: 0.9006 - val_loss: 0.0269 - val_accuracy: 0.9931
Epoch 8/

In [87]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
62/62 [==============================] - 89s 1s/step - loss: 1.1992 - accuracy: 0.7258 - val_loss: 0.0897 - val_accuracy: 0.9865
Epoch 2/10
62/62 [==============================] - 85s 1s/step - loss: 0.6802 - accuracy: 0.8118 - val_loss: 0.1277 - val_accuracy: 0.9788
Epoch 3/10
62/62 [==============================] - 84s 1s/step - loss: 0.5900 - accuracy: 0.8302 - val_loss: 0.0688 - val_accuracy: 0.9857
Epoch 4/10
62/62 [==============================] - 86s 1s/step - loss: 0.4690 - accuracy: 0.8671 - val_loss: 0.0904 - val_accuracy: 0.9763
Epoch 5/10
62/62 [==============================] - 86s 1s/step - loss: 0.4094 - accuracy: 0.8835 - val_loss: 0.0362 - val_accuracy: 0.9923
Epoch 6/10
62/62 [==============================] - 86s 1s/step - loss: 0.3598 - accuracy: 0.8956 - val_loss: 0.0789 - val_accuracy: 0.9852
Epoch 7/10
62/62 [==============================] - 85s 1s/step - loss: 0.3137 - accuracy: 0.9060 - val_loss: 0.0381 - val_accuracy: 0.9906
Epoch 8/10
62/62 

In [88]:
#new_model_dsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_dsa_e1"
new_model_dsa_dir  = "D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2"


i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dsa_e2_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [89]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [90]:
import gc
gc.collect()

499060

## Uncertainty

In [91]:
#dg_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/deep_gini_values.npy"

#deep_gini_values = np.load(dg_direction)
dg_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)[len(x_train):] 
print(len(deep_gini_values))

3921


In [92]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_adversary_training)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_adversary_training)

In [93]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  190
190
1 :
0  ->  380
380
2 :
0  ->  570
570
3 :
0  ->  760
760
4 :
0  ->  950
950
5 :
0  ->  1140
1140
6 :
0  ->  1330
1330
7 :
0  ->  1520
1520
8 :
0  ->  1710
1710
9 :
0  ->  1900
1900
10 :
0  ->  2090
2090
11 :
0  ->  2280
2280
12 :
0  ->  2470
2470
13 :
0  ->  2660
2660
14 :
0  ->  2850
2850
15 :
0  ->  3040
3040
16 :
0  ->  3230
3230
17 :
0  ->  3420
3420
18 :
0  ->  3610
3610
19 :
Last
0  ->  3921
3921


In [94]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [95]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

0
Epoch 1/10
3/3 [==============================] - 23s 9s/step - loss: 4.0973 - accuracy: 0.1147 - val_loss: 0.6378 - val_accuracy: 0.9074
Epoch 2/10
3/3 [==============================] - 17s 8s/step - loss: 3.0444 - accuracy: 0.2435 - val_loss: 0.3353 - val_accuracy: 0.9395
Epoch 3/10
3/3 [==============================] - 18s 8s/step - loss: 2.4542 - accuracy: 0.4038 - val_loss: 0.3711 - val_accuracy: 0.9025
Epoch 4/10
3/3 [==============================] - 18s 8s/step - loss: 2.2608 - accuracy: 0.4386 - val_loss: 2.9048 - val_accuracy: 0.7321
Epoch 5/10
3/3 [==============================] - 17s 8s/step - loss: 2.7180 - accuracy: 0.4397 - val_loss: 0.3935 - val_accuracy: 0.9056
Epoch 6/10
3/3 [==============================] - 17s 8s/step - loss: 1.9104 - accuracy: 0.4935 - val_loss: 0.2279 - val_accuracy: 0.9421
Epoch 7/10
3/3 [==============================] - 16s 8s/step - loss: 1.6760 - accuracy: 0.5661 - val_loss: 0.4955 - val_accuracy: 0.8709
Epoch 8/10
3/3 [================

In [96]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

1
Epoch 1/10
6/6 [==============================] - 28s 4s/step - loss: 3.9580 - accuracy: 0.1150 - val_loss: 0.6423 - val_accuracy: 0.9102
Epoch 2/10
6/6 [==============================] - 23s 4s/step - loss: 2.6658 - accuracy: 0.3023 - val_loss: 1.1063 - val_accuracy: 0.7742
Epoch 3/10
6/6 [==============================] - 23s 4s/step - loss: 2.2665 - accuracy: 0.4263 - val_loss: 0.6016 - val_accuracy: 0.8410
Epoch 4/10
6/6 [==============================] - 23s 4s/step - loss: 1.8054 - accuracy: 0.4972 - val_loss: 0.3815 - val_accuracy: 0.8969
Epoch 5/10
6/6 [==============================] - 23s 4s/step - loss: 1.7173 - accuracy: 0.4910 - val_loss: 0.3328 - val_accuracy: 0.9168
Epoch 6/10
6/6 [==============================] - 23s 4s/step - loss: 1.5441 - accuracy: 0.5665 - val_loss: 0.6626 - val_accuracy: 0.8339
Epoch 7/10
6/6 [==============================] - 23s 4s/step - loss: 1.2728 - accuracy: 0.6661 - val_loss: 0.5023 - val_accuracy: 0.8732
Epoch 8/10
6/6 [================

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

2
Epoch 1/10
9/9 [==============================] - 31s 3s/step - loss: 3.7407 - accuracy: 0.1637 - val_loss: 0.1991 - val_accuracy: 0.9594
Epoch 2/10
9/9 [==============================] - 25s 3s/step - loss: 2.3650 - accuracy: 0.3758 - val_loss: 0.3675 - val_accuracy: 0.9079
Epoch 3/10
9/9 [==============================] - 25s 3s/step - loss: 1.8873 - accuracy: 0.4794 - val_loss: 0.4138 - val_accuracy: 0.9010
Epoch 4/10
9/9 [==============================] - 26s 3s/step - loss: 1.6276 - accuracy: 0.5488 - val_loss: 0.4182 - val_accuracy: 0.8875
Epoch 5/10
9/9 [==============================] - 25s 3s/step - loss: 1.3009 - accuracy: 0.6044 - val_loss: 0.6079 - val_accuracy: 0.8479
Epoch 6/10
9/9 [==============================] - 25s 3s/step - loss: 1.2645 - accuracy: 0.6115 - val_loss: 0.4429 - val_accuracy: 0.8824
Epoch 7/10
9/9 [==============================] - 25s 3s/step - loss: 0.9863 - accuracy: 0.6848 - val_loss: 0.5070 - val_accuracy: 0.8729
Epoch 8/10
9/9 [================

In [98]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

3
Epoch 1/10
12/12 [==============================] - 35s 3s/step - loss: 4.0610 - accuracy: 0.1068 - val_loss: 0.1895 - val_accuracy: 0.9559
Epoch 2/10
12/12 [==============================] - 30s 3s/step - loss: 2.4772 - accuracy: 0.3388 - val_loss: 0.3459 - val_accuracy: 0.9298
Epoch 3/10
12/12 [==============================] - 30s 3s/step - loss: 1.9247 - accuracy: 0.4939 - val_loss: 0.5223 - val_accuracy: 0.8584
Epoch 4/10
12/12 [==============================] - 30s 3s/step - loss: 1.5911 - accuracy: 0.5965 - val_loss: 0.5320 - val_accuracy: 0.8538
Epoch 5/10
12/12 [==============================] - 31s 3s/step - loss: 1.3534 - accuracy: 0.6420 - val_loss: 0.5284 - val_accuracy: 0.8696
Epoch 6/10
12/12 [==============================] - 31s 3s/step - loss: 1.2139 - accuracy: 0.6637 - val_loss: 0.3819 - val_accuracy: 0.8931
Epoch 7/10
12/12 [==============================] - 31s 3s/step - loss: 1.0417 - accuracy: 0.7015 - val_loss: 0.6962 - val_accuracy: 0.8270
Epoch 8/10
12/12 [

In [99]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

4
Epoch 1/10
15/15 [==============================] - 40s 3s/step - loss: 3.7003 - accuracy: 0.1426 - val_loss: 0.2557 - val_accuracy: 0.9398
Epoch 2/10
15/15 [==============================] - 36s 3s/step - loss: 2.3048 - accuracy: 0.3907 - val_loss: 0.3231 - val_accuracy: 0.9199
Epoch 3/10
15/15 [==============================] - 38s 3s/step - loss: 1.7476 - accuracy: 0.5182 - val_loss: 0.3613 - val_accuracy: 0.8984
Epoch 4/10
15/15 [==============================] - 37s 3s/step - loss: 1.4775 - accuracy: 0.5731 - val_loss: 0.4162 - val_accuracy: 0.8941
Epoch 5/10
15/15 [==============================] - 37s 3s/step - loss: 1.3232 - accuracy: 0.6317 - val_loss: 0.4809 - val_accuracy: 0.8607
Epoch 6/10
15/15 [==============================] - 37s 3s/step - loss: 1.1601 - accuracy: 0.6750 - val_loss: 0.5155 - val_accuracy: 0.8625
Epoch 7/10
15/15 [==============================] - 38s 3s/step - loss: 1.0247 - accuracy: 0.7057 - val_loss: 0.3920 - val_accuracy: 0.9097
Epoch 8/10
15/15 [

In [100]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

5
Epoch 1/10
18/18 [==============================] - 45s 2s/step - loss: 3.2804 - accuracy: 0.2106 - val_loss: 0.1382 - val_accuracy: 0.9650
Epoch 2/10
18/18 [==============================] - 41s 2s/step - loss: 1.9200 - accuracy: 0.4586 - val_loss: 0.1061 - val_accuracy: 0.9686
Epoch 3/10
18/18 [==============================] - 43s 2s/step - loss: 1.5856 - accuracy: 0.5542 - val_loss: 0.1437 - val_accuracy: 0.9617
Epoch 4/10
18/18 [==============================] - 42s 2s/step - loss: 1.4034 - accuracy: 0.6022 - val_loss: 0.1449 - val_accuracy: 0.9622
Epoch 5/10
18/18 [==============================] - 41s 2s/step - loss: 1.1660 - accuracy: 0.6687 - val_loss: 0.1570 - val_accuracy: 0.9599
Epoch 6/10
18/18 [==============================] - 42s 2s/step - loss: 1.0356 - accuracy: 0.6783 - val_loss: 0.1401 - val_accuracy: 0.9604
Epoch 7/10
18/18 [==============================] - 42s 2s/step - loss: 0.8839 - accuracy: 0.7363 - val_loss: 0.1581 - val_accuracy: 0.9587
Epoch 8/10
18/18 [

In [101]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

6
Epoch 1/10
21/21 [==============================] - 48s 2s/step - loss: 3.2827 - accuracy: 0.2249 - val_loss: 0.1008 - val_accuracy: 0.9747
Epoch 2/10
21/21 [==============================] - 45s 2s/step - loss: 1.8793 - accuracy: 0.5083 - val_loss: 0.0817 - val_accuracy: 0.9786
Epoch 3/10
21/21 [==============================] - 44s 2s/step - loss: 1.5294 - accuracy: 0.5870 - val_loss: 0.0780 - val_accuracy: 0.9763
Epoch 4/10
21/21 [==============================] - 45s 2s/step - loss: 1.2979 - accuracy: 0.6199 - val_loss: 0.0884 - val_accuracy: 0.9786
Epoch 5/10
21/21 [==============================] - 44s 2s/step - loss: 1.1505 - accuracy: 0.6760 - val_loss: 0.1108 - val_accuracy: 0.9707
Epoch 6/10
21/21 [==============================] - 46s 2s/step - loss: 0.9294 - accuracy: 0.7283 - val_loss: 0.0785 - val_accuracy: 0.9775
Epoch 7/10
21/21 [==============================] - 45s 2s/step - loss: 0.8995 - accuracy: 0.7325 - val_loss: 0.0706 - val_accuracy: 0.9804
Epoch 8/10
21/21 [

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

7
Epoch 1/10
24/24 [==============================] - 52s 2s/step - loss: 2.9384 - accuracy: 0.3031 - val_loss: 0.0944 - val_accuracy: 0.9832
Epoch 2/10
24/24 [==============================] - 49s 2s/step - loss: 1.7035 - accuracy: 0.5051 - val_loss: 0.0507 - val_accuracy: 0.9872
Epoch 3/10
24/24 [==============================] - 50s 2s/step - loss: 1.4298 - accuracy: 0.5932 - val_loss: 0.0505 - val_accuracy: 0.9880
Epoch 4/10
24/24 [==============================] - 50s 2s/step - loss: 1.0710 - accuracy: 0.6857 - val_loss: 0.0493 - val_accuracy: 0.9865
Epoch 5/10
24/24 [==============================] - 49s 2s/step - loss: 0.9520 - accuracy: 0.7169 - val_loss: 0.0561 - val_accuracy: 0.9857
Epoch 6/10
24/24 [==============================] - 49s 2s/step - loss: 0.7976 - accuracy: 0.7563 - val_loss: 0.0678 - val_accuracy: 0.9796
Epoch 7/10
24/24 [==============================] - 49s 2s/step - loss: 0.7798 - accuracy: 0.7744 - val_loss: 0.0594 - val_accuracy: 0.9834
Epoch 8/10
24/24 [

In [103]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

8
Epoch 1/10
27/27 [==============================] - 58s 2s/step - loss: 2.6307 - accuracy: 0.3556 - val_loss: 0.0367 - val_accuracy: 0.9900
Epoch 2/10
27/27 [==============================] - 52s 2s/step - loss: 1.6364 - accuracy: 0.5639 - val_loss: 0.0418 - val_accuracy: 0.9893
Epoch 3/10
27/27 [==============================] - 54s 2s/step - loss: 1.2331 - accuracy: 0.6496 - val_loss: 0.0439 - val_accuracy: 0.9895
Epoch 4/10
27/27 [==============================] - 53s 2s/step - loss: 1.0754 - accuracy: 0.6831 - val_loss: 0.0622 - val_accuracy: 0.9826
Epoch 5/10
27/27 [==============================] - 55s 2s/step - loss: 0.8468 - accuracy: 0.7427 - val_loss: 0.0618 - val_accuracy: 0.9870
Epoch 6/10
27/27 [==============================] - 55s 2s/step - loss: 0.7691 - accuracy: 0.7800 - val_loss: 0.0561 - val_accuracy: 0.9867
Epoch 7/10
27/27 [==============================] - 55s 2s/step - loss: 0.6800 - accuracy: 0.7977 - val_loss: 0.0381 - val_accuracy: 0.9906
Epoch 8/10
27/27 [

In [104]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

9
Epoch 1/10
30/30 [==============================] - 63s 2s/step - loss: 2.6218 - accuracy: 0.3939 - val_loss: 0.0486 - val_accuracy: 0.9883
Epoch 2/10
30/30 [==============================] - 60s 2s/step - loss: 1.3874 - accuracy: 0.6141 - val_loss: 0.0478 - val_accuracy: 0.9888
Epoch 3/10
30/30 [==============================] - 59s 2s/step - loss: 1.2386 - accuracy: 0.6526 - val_loss: 0.0570 - val_accuracy: 0.9857
Epoch 4/10
30/30 [==============================] - 58s 2s/step - loss: 0.9817 - accuracy: 0.7256 - val_loss: 0.0392 - val_accuracy: 0.9895
Epoch 5/10
30/30 [==============================] - 60s 2s/step - loss: 0.8129 - accuracy: 0.7606 - val_loss: 0.0549 - val_accuracy: 0.9857
Epoch 6/10
30/30 [==============================] - 60s 2s/step - loss: 0.7788 - accuracy: 0.7779 - val_loss: 0.0568 - val_accuracy: 0.9839
Epoch 7/10
30/30 [==============================] - 57s 2s/step - loss: 0.6271 - accuracy: 0.8154 - val_loss: 0.0520 - val_accuracy: 0.9844
Epoch 8/10
30/30 [

In [105]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

10
Epoch 1/10
33/33 [==============================] - 70s 2s/step - loss: 2.1706 - accuracy: 0.5071 - val_loss: 0.0368 - val_accuracy: 0.9908
Epoch 2/10
33/33 [==============================] - 65s 2s/step - loss: 1.2302 - accuracy: 0.6635 - val_loss: 0.0741 - val_accuracy: 0.9809
Epoch 3/10
33/33 [==============================] - 66s 2s/step - loss: 1.0893 - accuracy: 0.6834 - val_loss: 0.0417 - val_accuracy: 0.9898
Epoch 4/10
33/33 [==============================] - 66s 2s/step - loss: 0.8455 - accuracy: 0.7449 - val_loss: 0.0808 - val_accuracy: 0.9819
Epoch 5/10
33/33 [==============================] - 63s 2s/step - loss: 0.6725 - accuracy: 0.8049 - val_loss: 0.0372 - val_accuracy: 0.9918
Epoch 6/10
33/33 [==============================] - 65s 2s/step - loss: 0.6485 - accuracy: 0.8108 - val_loss: 0.0430 - val_accuracy: 0.9895
Epoch 7/10
33/33 [==============================] - 65s 2s/step - loss: 0.5989 - accuracy: 0.8082 - val_loss: 0.0370 - val_accuracy: 0.9906
Epoch 8/10
33/33 

In [106]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

11
Epoch 1/10
36/36 [==============================] - 76s 2s/step - loss: 1.8773 - accuracy: 0.5507 - val_loss: 0.0475 - val_accuracy: 0.9878
Epoch 2/10
36/36 [==============================] - 70s 2s/step - loss: 1.1114 - accuracy: 0.6865 - val_loss: 0.0423 - val_accuracy: 0.9903
Epoch 3/10
36/36 [==============================] - 73s 2s/step - loss: 0.9256 - accuracy: 0.7277 - val_loss: 0.0422 - val_accuracy: 0.9893
Epoch 4/10
36/36 [==============================] - 73s 2s/step - loss: 0.8050 - accuracy: 0.7667 - val_loss: 0.0346 - val_accuracy: 0.9931
Epoch 5/10
36/36 [==============================] - 72s 2s/step - loss: 0.6853 - accuracy: 0.7998 - val_loss: 0.0442 - val_accuracy: 0.9906
Epoch 6/10
36/36 [==============================] - 73s 2s/step - loss: 0.5855 - accuracy: 0.8265 - val_loss: 0.0403 - val_accuracy: 0.9895
Epoch 7/10
36/36 [==============================] - 71s 2s/step - loss: 0.5178 - accuracy: 0.8457 - val_loss: 0.0420 - val_accuracy: 0.9875
Epoch 8/10
36/36 

In [107]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

12
Epoch 1/10
39/39 [==============================] - 86s 2s/step - loss: 1.9143 - accuracy: 0.5495 - val_loss: 0.0579 - val_accuracy: 0.9852
Epoch 2/10
39/39 [==============================] - 81s 2s/step - loss: 1.1206 - accuracy: 0.7000 - val_loss: 0.0370 - val_accuracy: 0.9911
Epoch 3/10
39/39 [==============================] - 82s 2s/step - loss: 0.8823 - accuracy: 0.7504 - val_loss: 0.0418 - val_accuracy: 0.9900
Epoch 4/10
39/39 [==============================] - 81s 2s/step - loss: 0.7565 - accuracy: 0.7847 - val_loss: 0.0360 - val_accuracy: 0.9913
Epoch 5/10
39/39 [==============================] - 83s 2s/step - loss: 0.6408 - accuracy: 0.8207 - val_loss: 0.0548 - val_accuracy: 0.9849
Epoch 6/10
39/39 [==============================] - 80s 2s/step - loss: 0.5842 - accuracy: 0.8285 - val_loss: 0.0393 - val_accuracy: 0.9895
Epoch 7/10
39/39 [==============================] - 78s 2s/step - loss: 0.5559 - accuracy: 0.8409 - val_loss: 0.0406 - val_accuracy: 0.9893
Epoch 8/10
39/39 

In [108]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

13
Epoch 1/10
42/42 [==============================] - 97s 2s/step - loss: 1.8268 - accuracy: 0.5726 - val_loss: 0.0292 - val_accuracy: 0.9926
Epoch 2/10
42/42 [==============================] - 93s 2s/step - loss: 1.0148 - accuracy: 0.7232 - val_loss: 0.0493 - val_accuracy: 0.9878
Epoch 3/10
42/42 [==============================] - 94s 2s/step - loss: 0.7865 - accuracy: 0.7712 - val_loss: 0.0490 - val_accuracy: 0.9895
Epoch 4/10
42/42 [==============================] - 92s 2s/step - loss: 0.6291 - accuracy: 0.8056 - val_loss: 0.0421 - val_accuracy: 0.9890
Epoch 5/10
42/42 [==============================] - 91s 2s/step - loss: 0.5802 - accuracy: 0.8334 - val_loss: 0.0506 - val_accuracy: 0.9885
Epoch 6/10
42/42 [==============================] - 90s 2s/step - loss: 0.5534 - accuracy: 0.8374 - val_loss: 0.0549 - val_accuracy: 0.9875
Epoch 7/10
42/42 [==============================] - 89s 2s/step - loss: 0.4722 - accuracy: 0.8596 - val_loss: 0.0431 - val_accuracy: 0.9878
Epoch 8/10
42/42 

In [109]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

14
Epoch 1/10
45/45 [==============================] - 106s 2s/step - loss: 1.7131 - accuracy: 0.6184 - val_loss: 0.0344 - val_accuracy: 0.9918
Epoch 2/10
45/45 [==============================] - 111s 2s/step - loss: 0.9326 - accuracy: 0.7518 - val_loss: 0.0563 - val_accuracy: 0.9867
Epoch 3/10
45/45 [==============================] - 108s 2s/step - loss: 0.7257 - accuracy: 0.7999 - val_loss: 0.0529 - val_accuracy: 0.9865
Epoch 4/10
45/45 [==============================] - 106s 2s/step - loss: 0.6477 - accuracy: 0.8100 - val_loss: 0.1604 - val_accuracy: 0.9681
Epoch 5/10
45/45 [==============================] - 104s 2s/step - loss: 0.5435 - accuracy: 0.8379 - val_loss: 0.0291 - val_accuracy: 0.9936
Epoch 6/10
45/45 [==============================] - 103s 2s/step - loss: 0.4598 - accuracy: 0.8594 - val_loss: 0.0532 - val_accuracy: 0.9888
Epoch 7/10
45/45 [==============================] - 107s 2s/step - loss: 0.4161 - accuracy: 0.8801 - val_loss: 0.0319 - val_accuracy: 0.9911
Epoch 8/10

In [110]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

15
Epoch 1/10
48/48 [==============================] - 118s 2s/step - loss: 1.4954 - accuracy: 0.6437 - val_loss: 0.0381 - val_accuracy: 0.9893
Epoch 2/10
48/48 [==============================] - 110s 2s/step - loss: 0.8859 - accuracy: 0.7521 - val_loss: 0.0958 - val_accuracy: 0.9816
Epoch 3/10
48/48 [==============================] - 114s 2s/step - loss: 0.7363 - accuracy: 0.8000 - val_loss: 0.0749 - val_accuracy: 0.9842
Epoch 4/10
48/48 [==============================] - 119s 3s/step - loss: 0.6063 - accuracy: 0.8231 - val_loss: 0.0715 - val_accuracy: 0.9832
Epoch 5/10
48/48 [==============================] - 116s 2s/step - loss: 0.5137 - accuracy: 0.8494 - val_loss: 0.0479 - val_accuracy: 0.9870
Epoch 6/10
48/48 [==============================] - 115s 2s/step - loss: 0.4539 - accuracy: 0.8592 - val_loss: 0.0385 - val_accuracy: 0.9906
Epoch 7/10
48/48 [==============================] - 122s 3s/step - loss: 0.3852 - accuracy: 0.8749 - val_loss: 0.0492 - val_accuracy: 0.9870
Epoch 8/10

In [111]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

16
Epoch 1/10
51/51 [==============================] - 141s 3s/step - loss: 1.4409 - accuracy: 0.6719 - val_loss: 0.0373 - val_accuracy: 0.9906
Epoch 2/10
51/51 [==============================] - 131s 3s/step - loss: 0.7759 - accuracy: 0.7875 - val_loss: 0.0468 - val_accuracy: 0.9857
Epoch 3/10
51/51 [==============================] - 127s 2s/step - loss: 0.6183 - accuracy: 0.8210 - val_loss: 0.0470 - val_accuracy: 0.9888
Epoch 4/10
51/51 [==============================] - 132s 3s/step - loss: 0.5240 - accuracy: 0.8453 - val_loss: 0.0556 - val_accuracy: 0.9885
Epoch 5/10
51/51 [==============================] - 129s 3s/step - loss: 0.4836 - accuracy: 0.8610 - val_loss: 0.0429 - val_accuracy: 0.9903
Epoch 6/10
51/51 [==============================] - 129s 3s/step - loss: 0.4378 - accuracy: 0.8627 - val_loss: 0.0614 - val_accuracy: 0.9865
Epoch 7/10
51/51 [==============================] - 132s 3s/step - loss: 0.3829 - accuracy: 0.8870 - val_loss: 0.0397 - val_accuracy: 0.9913
Epoch 8/10

In [112]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

17
Epoch 1/10
54/54 [==============================] - 157s 3s/step - loss: 1.4192 - accuracy: 0.6677 - val_loss: 0.0603 - val_accuracy: 0.9855
Epoch 2/10
54/54 [==============================] - 147s 3s/step - loss: 0.7892 - accuracy: 0.7798 - val_loss: 0.0306 - val_accuracy: 0.9911
Epoch 3/10
54/54 [==============================] - 152s 3s/step - loss: 0.6417 - accuracy: 0.8174 - val_loss: 0.0499 - val_accuracy: 0.9847
Epoch 4/10
54/54 [==============================] - 148s 3s/step - loss: 0.5486 - accuracy: 0.8451 - val_loss: 0.0313 - val_accuracy: 0.9900
Epoch 5/10
54/54 [==============================] - 152s 3s/step - loss: 0.4175 - accuracy: 0.8715 - val_loss: 0.0459 - val_accuracy: 0.9883
Epoch 6/10
54/54 [==============================] - 144s 3s/step - loss: 0.3878 - accuracy: 0.8785 - val_loss: 0.0437 - val_accuracy: 0.9888
Epoch 7/10
54/54 [==============================] - 146s 3s/step - loss: 0.3661 - accuracy: 0.8905 - val_loss: 0.0290 - val_accuracy: 0.9929
Epoch 8/10

In [113]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

18
Epoch 1/10
57/57 [==============================] - 167s 3s/step - loss: 1.3415 - accuracy: 0.6940 - val_loss: 0.0403 - val_accuracy: 0.9906
Epoch 2/10
57/57 [==============================] - 163s 3s/step - loss: 0.7402 - accuracy: 0.8007 - val_loss: 0.0444 - val_accuracy: 0.9878
Epoch 3/10
57/57 [==============================] - 156s 3s/step - loss: 0.6256 - accuracy: 0.8201 - val_loss: 0.0849 - val_accuracy: 0.9801
Epoch 4/10
57/57 [==============================] - 153s 3s/step - loss: 0.5248 - accuracy: 0.8521 - val_loss: 0.0671 - val_accuracy: 0.9847
Epoch 5/10
57/57 [==============================] - 158s 3s/step - loss: 0.4268 - accuracy: 0.8770 - val_loss: 0.0381 - val_accuracy: 0.9903
Epoch 6/10
57/57 [==============================] - 162s 3s/step - loss: 0.3696 - accuracy: 0.8955 - val_loss: 0.0598 - val_accuracy: 0.9832
Epoch 7/10
57/57 [==============================] - 160s 3s/step - loss: 0.3551 - accuracy: 0.8942 - val_loss: 0.0402 - val_accuracy: 0.9893
Epoch 8/10

In [114]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

19
Epoch 1/10
62/62 [==============================] - 185s 3s/step - loss: 1.1404 - accuracy: 0.7255 - val_loss: 0.0487 - val_accuracy: 0.9870
Epoch 2/10
62/62 [==============================] - 192s 3s/step - loss: 0.6975 - accuracy: 0.8082 - val_loss: 0.0629 - val_accuracy: 0.9798
Epoch 3/10
62/62 [==============================] - 186s 3s/step - loss: 0.5691 - accuracy: 0.8486 - val_loss: 0.0416 - val_accuracy: 0.9893
Epoch 4/10
62/62 [==============================] - 186s 3s/step - loss: 0.4431 - accuracy: 0.8724 - val_loss: 0.0879 - val_accuracy: 0.9826
Epoch 5/10
62/62 [==============================] - 181s 3s/step - loss: 0.3967 - accuracy: 0.8860 - val_loss: 0.0512 - val_accuracy: 0.9870
Epoch 6/10
62/62 [==============================] - 181s 3s/step - loss: 0.3712 - accuracy: 0.8919 - val_loss: 0.0323 - val_accuracy: 0.9923
Epoch 7/10
62/62 [==============================] - 189s 3s/step - loss: 0.2860 - accuracy: 0.9166 - val_loss: 0.0386 - val_accuracy: 0.9903
Epoch 8/10

In [115]:
#new_model_dg_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_dg_e1"
new_model_dg_dir  = "D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_dg_e2_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug

In [116]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [117]:

gc.collect()

528874

### Softmax 

In [118]:
#softmax values
#se_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/softmax_values.npy"
se_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"

se_values = np.load(se_direction)[len(x_train):] 
print(len(se_values))


3921


In [119]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_adversary_training)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_adversary_training)

In [120]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  190
190
1 :
0  ->  380
380
2 :
0  ->  570
570
3 :
0  ->  760
760
4 :
0  ->  950
950
5 :
0  ->  1140
1140
6 :
0  ->  1330
1330
7 :
0  ->  1520
1520
8 :
0  ->  1710
1710
9 :
0  ->  1900
1900
10 :
0  ->  2090
2090
11 :
0  ->  2280
2280
12 :
0  ->  2470
2470
13 :
0  ->  2660
2660
14 :
0  ->  2850
2850
15 :
0  ->  3040
3040
16 :
0  ->  3230
3230
17 :
0  ->  3420
3420
18 :
0  ->  3610
3610
19 :
Last
0  ->  3921
3921


In [121]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [122]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

0
Epoch 1/10
3/3 [==============================] - 18s 7s/step - loss: 4.3091 - accuracy: 0.1042 - val_loss: 0.3625 - val_accuracy: 0.9382
Epoch 2/10
3/3 [==============================] - 14s 7s/step - loss: 3.1700 - accuracy: 0.1768 - val_loss: 0.2955 - val_accuracy: 0.9650
Epoch 3/10
3/3 [==============================] - 14s 7s/step - loss: 2.8160 - accuracy: 0.2866 - val_loss: 0.1375 - val_accuracy: 0.9689
Epoch 4/10
3/3 [==============================] - 15s 7s/step - loss: 2.3320 - accuracy: 0.3966 - val_loss: 0.1574 - val_accuracy: 0.9630
Epoch 5/10
3/3 [==============================] - 15s 7s/step - loss: 1.9758 - accuracy: 0.5058 - val_loss: 0.1633 - val_accuracy: 0.9584
Epoch 6/10
3/3 [==============================] - 15s 7s/step - loss: 1.7361 - accuracy: 0.5204 - val_loss: 0.2001 - val_accuracy: 0.9518
Epoch 7/10
3/3 [==============================] - 15s 7s/step - loss: 1.6302 - accuracy: 0.5237 - val_loss: 0.2054 - val_accuracy: 0.9454
Epoch 8/10
3/3 [================

In [123]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

1
Epoch 1/10
6/6 [==============================] - 25s 4s/step - loss: 3.9600 - accuracy: 0.0993 - val_loss: 0.8803 - val_accuracy: 0.8280
Epoch 2/10
6/6 [==============================] - 22s 4s/step - loss: 2.4572 - accuracy: 0.3509 - val_loss: 0.2314 - val_accuracy: 0.9551
Epoch 3/10
6/6 [==============================] - 21s 4s/step - loss: 1.9151 - accuracy: 0.4561 - val_loss: 0.6457 - val_accuracy: 0.8640
Epoch 4/10
6/6 [==============================] - 21s 4s/step - loss: 1.5533 - accuracy: 0.5876 - val_loss: 0.4444 - val_accuracy: 0.9028
Epoch 5/10
6/6 [==============================] - 21s 4s/step - loss: 1.4788 - accuracy: 0.5951 - val_loss: 0.4031 - val_accuracy: 0.9079
Epoch 6/10
6/6 [==============================] - 21s 4s/step - loss: 1.2750 - accuracy: 0.6632 - val_loss: 0.5179 - val_accuracy: 0.8714
Epoch 7/10
6/6 [==============================] - 21s 4s/step - loss: 1.1474 - accuracy: 0.6743 - val_loss: 0.4601 - val_accuracy: 0.8933
Epoch 8/10
6/6 [================

In [124]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

2
Epoch 1/10
9/9 [==============================] - 31s 3s/step - loss: 3.8541 - accuracy: 0.1229 - val_loss: 0.2678 - val_accuracy: 0.9334
Epoch 2/10
9/9 [==============================] - 27s 3s/step - loss: 2.5668 - accuracy: 0.3399 - val_loss: 0.4138 - val_accuracy: 0.8967
Epoch 3/10
9/9 [==============================] - 26s 3s/step - loss: 2.0776 - accuracy: 0.4666 - val_loss: 0.3545 - val_accuracy: 0.9069
Epoch 4/10
9/9 [==============================] - 26s 3s/step - loss: 1.5869 - accuracy: 0.5659 - val_loss: 0.2990 - val_accuracy: 0.9352
Epoch 5/10
9/9 [==============================] - 26s 3s/step - loss: 1.5999 - accuracy: 0.5892 - val_loss: 0.2563 - val_accuracy: 0.9342
Epoch 6/10
9/9 [==============================] - 26s 3s/step - loss: 1.2341 - accuracy: 0.6401 - val_loss: 0.2961 - val_accuracy: 0.9189
Epoch 7/10
9/9 [==============================] - 26s 3s/step - loss: 1.0766 - accuracy: 0.6716 - val_loss: 0.4027 - val_accuracy: 0.8916
Epoch 8/10
9/9 [================

In [125]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

3
Epoch 1/10
12/12 [==============================] - 36s 3s/step - loss: 3.8704 - accuracy: 0.1227 - val_loss: 0.4707 - val_accuracy: 0.9331
Epoch 2/10
12/12 [==============================] - 32s 3s/step - loss: 2.4541 - accuracy: 0.3443 - val_loss: 0.3044 - val_accuracy: 0.9237
Epoch 3/10
12/12 [==============================] - 32s 3s/step - loss: 1.8461 - accuracy: 0.4809 - val_loss: 0.3065 - val_accuracy: 0.9183
Epoch 4/10
12/12 [==============================] - 32s 3s/step - loss: 1.4759 - accuracy: 0.5784 - val_loss: 0.3456 - val_accuracy: 0.9148
Epoch 5/10
12/12 [==============================] - 32s 3s/step - loss: 1.4211 - accuracy: 0.5581 - val_loss: 0.4233 - val_accuracy: 0.8951
Epoch 6/10
12/12 [==============================] - 32s 3s/step - loss: 1.2255 - accuracy: 0.6269 - val_loss: 0.2926 - val_accuracy: 0.9196
Epoch 7/10
12/12 [==============================] - 32s 3s/step - loss: 0.9371 - accuracy: 0.7183 - val_loss: 0.3832 - val_accuracy: 0.9066
Epoch 8/10
12/12 [

In [126]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

4
Epoch 1/10
15/15 [==============================] - 41s 3s/step - loss: 3.5081 - accuracy: 0.1836 - val_loss: 0.4724 - val_accuracy: 0.8798
Epoch 2/10
15/15 [==============================] - 37s 3s/step - loss: 2.1401 - accuracy: 0.4297 - val_loss: 0.2419 - val_accuracy: 0.9342
Epoch 3/10
15/15 [==============================] - 37s 3s/step - loss: 1.8169 - accuracy: 0.4926 - val_loss: 0.1834 - val_accuracy: 0.9564
Epoch 4/10
15/15 [==============================] - 37s 3s/step - loss: 1.3643 - accuracy: 0.6227 - val_loss: 0.2155 - val_accuracy: 0.9400
Epoch 5/10
15/15 [==============================] - 37s 3s/step - loss: 1.3272 - accuracy: 0.6313 - val_loss: 0.3116 - val_accuracy: 0.9094
Epoch 6/10
15/15 [==============================] - 37s 3s/step - loss: 1.1210 - accuracy: 0.6588 - val_loss: 0.2778 - val_accuracy: 0.9212
Epoch 7/10
15/15 [==============================] - 36s 2s/step - loss: 0.9115 - accuracy: 0.7172 - val_loss: 0.3109 - val_accuracy: 0.9127
Epoch 8/10
15/15 [

In [127]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

5
Epoch 1/10
18/18 [==============================] - 47s 2s/step - loss: 3.4439 - accuracy: 0.1858 - val_loss: 0.1656 - val_accuracy: 0.9653
Epoch 2/10
18/18 [==============================] - 45s 3s/step - loss: 2.0488 - accuracy: 0.4363 - val_loss: 0.1609 - val_accuracy: 0.9699
Epoch 3/10
18/18 [==============================] - 43s 2s/step - loss: 1.7089 - accuracy: 0.5222 - val_loss: 0.5383 - val_accuracy: 0.8686
Epoch 4/10
18/18 [==============================] - 43s 2s/step - loss: 1.4399 - accuracy: 0.5679 - val_loss: 0.1679 - val_accuracy: 0.9612
Epoch 5/10
18/18 [==============================] - 43s 2s/step - loss: 1.0890 - accuracy: 0.6649 - val_loss: 0.2701 - val_accuracy: 0.9367
Epoch 6/10
18/18 [==============================] - 43s 2s/step - loss: 1.0401 - accuracy: 0.6935 - val_loss: 0.2036 - val_accuracy: 0.9518
Epoch 7/10
18/18 [==============================] - 45s 3s/step - loss: 0.9430 - accuracy: 0.7184 - val_loss: 0.1626 - val_accuracy: 0.9543
Epoch 8/10
18/18 [

In [128]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

6
Epoch 1/10
21/21 [==============================] - 54s 2s/step - loss: 3.3158 - accuracy: 0.2408 - val_loss: 0.0692 - val_accuracy: 0.9811
Epoch 2/10
21/21 [==============================] - 50s 2s/step - loss: 1.8383 - accuracy: 0.5151 - val_loss: 0.1217 - val_accuracy: 0.9661
Epoch 3/10
21/21 [==============================] - 51s 2s/step - loss: 1.4728 - accuracy: 0.5951 - val_loss: 0.0800 - val_accuracy: 0.9798
Epoch 4/10
21/21 [==============================] - 53s 3s/step - loss: 1.1448 - accuracy: 0.6690 - val_loss: 0.1074 - val_accuracy: 0.9730
Epoch 5/10
21/21 [==============================] - 49s 2s/step - loss: 1.2005 - accuracy: 0.6552 - val_loss: 0.0892 - val_accuracy: 0.9781
Epoch 6/10
21/21 [==============================] - 49s 2s/step - loss: 0.9722 - accuracy: 0.7240 - val_loss: 0.1048 - val_accuracy: 0.9712
Epoch 7/10
21/21 [==============================] - 50s 2s/step - loss: 0.8397 - accuracy: 0.7345 - val_loss: 0.0921 - val_accuracy: 0.9740
Epoch 8/10
21/21 [

In [129]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

7
Epoch 1/10
24/24 [==============================] - 59s 2s/step - loss: 3.1300 - accuracy: 0.2662 - val_loss: 0.0492 - val_accuracy: 0.9880
Epoch 2/10
24/24 [==============================] - 54s 2s/step - loss: 1.6776 - accuracy: 0.5349 - val_loss: 0.0816 - val_accuracy: 0.9770
Epoch 3/10
24/24 [==============================] - 56s 2s/step - loss: 1.3735 - accuracy: 0.5908 - val_loss: 0.1701 - val_accuracy: 0.9673
Epoch 4/10
24/24 [==============================] - 56s 2s/step - loss: 1.0643 - accuracy: 0.6956 - val_loss: 0.0457 - val_accuracy: 0.9870
Epoch 5/10
24/24 [==============================] - 56s 2s/step - loss: 0.9945 - accuracy: 0.7184 - val_loss: 0.0447 - val_accuracy: 0.9893
Epoch 6/10
24/24 [==============================] - 54s 2s/step - loss: 0.8541 - accuracy: 0.7484 - val_loss: 0.0724 - val_accuracy: 0.9788
Epoch 7/10
24/24 [==============================] - 55s 2s/step - loss: 0.8340 - accuracy: 0.7464 - val_loss: 0.1071 - val_accuracy: 0.9694
Epoch 8/10
24/24 [

In [130]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

8
Epoch 1/10
27/27 [==============================] - 71s 3s/step - loss: 3.0092 - accuracy: 0.3351 - val_loss: 0.0480 - val_accuracy: 0.9878
Epoch 2/10
27/27 [==============================] - 66s 2s/step - loss: 1.4880 - accuracy: 0.5892 - val_loss: 0.0724 - val_accuracy: 0.9829
Epoch 3/10
27/27 [==============================] - 64s 2s/step - loss: 1.2039 - accuracy: 0.6681 - val_loss: 0.0523 - val_accuracy: 0.9875
Epoch 4/10
27/27 [==============================] - 62s 2s/step - loss: 1.0508 - accuracy: 0.6919 - val_loss: 0.0796 - val_accuracy: 0.9796
Epoch 5/10
27/27 [==============================] - 66s 2s/step - loss: 0.9562 - accuracy: 0.7084 - val_loss: 0.0452 - val_accuracy: 0.9888
Epoch 6/10
27/27 [==============================] - 69s 3s/step - loss: 0.7965 - accuracy: 0.7662 - val_loss: 0.0543 - val_accuracy: 0.9865
Epoch 7/10
27/27 [==============================] - 65s 2s/step - loss: 0.7541 - accuracy: 0.7606 - val_loss: 0.0459 - val_accuracy: 0.9898
Epoch 8/10
27/27 [

In [131]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

9
Epoch 1/10
30/30 [==============================] - 71s 2s/step - loss: 2.5904 - accuracy: 0.4162 - val_loss: 0.0533 - val_accuracy: 0.9862
Epoch 2/10
30/30 [==============================] - 68s 2s/step - loss: 1.4819 - accuracy: 0.5912 - val_loss: 0.0425 - val_accuracy: 0.9895
Epoch 3/10
30/30 [==============================] - 71s 2s/step - loss: 1.0833 - accuracy: 0.6991 - val_loss: 0.0412 - val_accuracy: 0.9895
Epoch 4/10
30/30 [==============================] - 72s 2s/step - loss: 0.9737 - accuracy: 0.7317 - val_loss: 0.0435 - val_accuracy: 0.9888
Epoch 5/10
30/30 [==============================] - 71s 2s/step - loss: 0.7732 - accuracy: 0.7825 - val_loss: 0.0351 - val_accuracy: 0.9906
Epoch 6/10
30/30 [==============================] - 72s 2s/step - loss: 0.7337 - accuracy: 0.7774 - val_loss: 0.0982 - val_accuracy: 0.9781
Epoch 7/10
30/30 [==============================] - 71s 2s/step - loss: 0.6817 - accuracy: 0.8029 - val_loss: 0.0364 - val_accuracy: 0.9913
Epoch 8/10
30/30 [

In [132]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

10
Epoch 1/10
33/33 [==============================] - 79s 2s/step - loss: 2.2346 - accuracy: 0.4721 - val_loss: 0.0603 - val_accuracy: 0.9844
Epoch 2/10
33/33 [==============================] - 71s 2s/step - loss: 1.2432 - accuracy: 0.6378 - val_loss: 0.0459 - val_accuracy: 0.9872
Epoch 3/10
33/33 [==============================] - 73s 2s/step - loss: 0.9992 - accuracy: 0.7080 - val_loss: 0.0521 - val_accuracy: 0.9875
Epoch 4/10
33/33 [==============================] - 80s 2s/step - loss: 0.7843 - accuracy: 0.7714 - val_loss: 0.0547 - val_accuracy: 0.9852
Epoch 5/10
33/33 [==============================] - 74s 2s/step - loss: 0.7105 - accuracy: 0.7909 - val_loss: 0.0353 - val_accuracy: 0.9908
Epoch 6/10
33/33 [==============================] - 77s 2s/step - loss: 0.6587 - accuracy: 0.8113 - val_loss: 0.0292 - val_accuracy: 0.9929
Epoch 7/10
33/33 [==============================] - 75s 2s/step - loss: 0.5210 - accuracy: 0.8301 - val_loss: 0.0325 - val_accuracy: 0.9921
Epoch 8/10
33/33 

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

11
Epoch 1/10
36/36 [==============================] - 85s 2s/step - loss: 2.0487 - accuracy: 0.4994 - val_loss: 0.0529 - val_accuracy: 0.9849
Epoch 2/10
36/36 [==============================] - 86s 2s/step - loss: 1.1235 - accuracy: 0.6830 - val_loss: 0.0503 - val_accuracy: 0.9875
Epoch 3/10
36/36 [==============================] - 92s 3s/step - loss: 0.9147 - accuracy: 0.7255 - val_loss: 0.0932 - val_accuracy: 0.9829
Epoch 4/10
36/36 [==============================] - 88s 2s/step - loss: 0.8206 - accuracy: 0.7621 - val_loss: 0.0342 - val_accuracy: 0.9923
Epoch 5/10
36/36 [==============================] - 83s 2s/step - loss: 0.6805 - accuracy: 0.8033 - val_loss: 0.0481 - val_accuracy: 0.9872
Epoch 6/10
36/36 [==============================] - 86s 2s/step - loss: 0.5921 - accuracy: 0.8146 - val_loss: 0.0577 - val_accuracy: 0.9852
Epoch 7/10
36/36 [==============================] - 84s 2s/step - loss: 0.5884 - accuracy: 0.8218 - val_loss: 0.0725 - val_accuracy: 0.9829
Epoch 8/10
36/36 

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

12
Epoch 1/10
39/39 [==============================] - 96s 2s/step - loss: 1.8885 - accuracy: 0.5567 - val_loss: 0.0464 - val_accuracy: 0.9903
Epoch 2/10
39/39 [==============================] - 93s 2s/step - loss: 1.0320 - accuracy: 0.7199 - val_loss: 0.0522 - val_accuracy: 0.9865
Epoch 3/10
39/39 [==============================] - 95s 2s/step - loss: 0.8807 - accuracy: 0.7562 - val_loss: 0.0406 - val_accuracy: 0.9900
Epoch 4/10
39/39 [==============================] - 93s 2s/step - loss: 0.7010 - accuracy: 0.7837 - val_loss: 0.0988 - val_accuracy: 0.9778
Epoch 5/10
39/39 [==============================] - 94s 2s/step - loss: 0.6164 - accuracy: 0.8170 - val_loss: 0.0616 - val_accuracy: 0.9865
Epoch 6/10
39/39 [==============================] - 90s 2s/step - loss: 0.5004 - accuracy: 0.8457 - val_loss: 0.0522 - val_accuracy: 0.9898
Epoch 7/10
39/39 [==============================] - 93s 2s/step - loss: 0.5218 - accuracy: 0.8477 - val_loss: 0.0568 - val_accuracy: 0.9842
Epoch 8/10
39/39 

In [135]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

13
Epoch 1/10
42/42 [==============================] - 108s 3s/step - loss: 1.6852 - accuracy: 0.5996 - val_loss: 0.0491 - val_accuracy: 0.9895
Epoch 2/10
42/42 [==============================] - 95s 2s/step - loss: 0.9371 - accuracy: 0.7351 - val_loss: 0.0419 - val_accuracy: 0.9893
Epoch 3/10
42/42 [==============================] - 115s 3s/step - loss: 0.8274 - accuracy: 0.7683 - val_loss: 0.0548 - val_accuracy: 0.9855
Epoch 4/10
42/42 [==============================] - 102s 2s/step - loss: 0.6822 - accuracy: 0.7991 - val_loss: 0.0407 - val_accuracy: 0.9906
Epoch 5/10
42/42 [==============================] - 100s 2s/step - loss: 0.6026 - accuracy: 0.8150 - val_loss: 0.0594 - val_accuracy: 0.9865
Epoch 6/10
42/42 [==============================] - 100s 2s/step - loss: 0.5354 - accuracy: 0.8329 - val_loss: 0.0348 - val_accuracy: 0.9929
Epoch 7/10
42/42 [==============================] - 106s 3s/step - loss: 0.4877 - accuracy: 0.8549 - val_loss: 0.0524 - val_accuracy: 0.9834
Epoch 8/10


In [136]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

14
Epoch 1/10
45/45 [==============================] - 124s 3s/step - loss: 1.6006 - accuracy: 0.6055 - val_loss: 0.0743 - val_accuracy: 0.9860
Epoch 2/10
45/45 [==============================] - 135s 3s/step - loss: 0.9267 - accuracy: 0.7437 - val_loss: 0.0412 - val_accuracy: 0.9888
Epoch 3/10
45/45 [==============================] - 123s 3s/step - loss: 0.7513 - accuracy: 0.7913 - val_loss: 0.0463 - val_accuracy: 0.9880
Epoch 4/10
45/45 [==============================] - 128s 3s/step - loss: 0.5948 - accuracy: 0.8324 - val_loss: 0.0535 - val_accuracy: 0.9875
Epoch 5/10
45/45 [==============================] - 118s 3s/step - loss: 0.5840 - accuracy: 0.8350 - val_loss: 0.0336 - val_accuracy: 0.9908
Epoch 6/10
45/45 [==============================] - 103s 2s/step - loss: 0.4540 - accuracy: 0.8703 - val_loss: 0.0412 - val_accuracy: 0.9898
Epoch 7/10
45/45 [==============================] - 101s 2s/step - loss: 0.4253 - accuracy: 0.8787 - val_loss: 0.0377 - val_accuracy: 0.9885
Epoch 8/10

In [137]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

15
Epoch 1/10
48/48 [==============================] - 124s 3s/step - loss: 1.5233 - accuracy: 0.6548 - val_loss: 0.0499 - val_accuracy: 0.9875
Epoch 2/10
48/48 [==============================] - 123s 3s/step - loss: 0.8670 - accuracy: 0.7613 - val_loss: 0.0411 - val_accuracy: 0.9890
Epoch 3/10
48/48 [==============================] - 125s 3s/step - loss: 0.6858 - accuracy: 0.7997 - val_loss: 0.0562 - val_accuracy: 0.9855
Epoch 4/10
48/48 [==============================] - 125s 3s/step - loss: 0.5822 - accuracy: 0.8329 - val_loss: 0.0307 - val_accuracy: 0.9921
Epoch 5/10
48/48 [==============================] - 123s 3s/step - loss: 0.5292 - accuracy: 0.8430 - val_loss: 0.0588 - val_accuracy: 0.9860
Epoch 6/10
48/48 [==============================] - 121s 3s/step - loss: 0.4921 - accuracy: 0.8573 - val_loss: 0.0719 - val_accuracy: 0.9832
Epoch 7/10
48/48 [==============================] - 122s 3s/step - loss: 0.4106 - accuracy: 0.8809 - val_loss: 0.0309 - val_accuracy: 0.9921
Epoch 8/10

In [138]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

16
Epoch 1/10
51/51 [==============================] - 124s 2s/step - loss: 1.4932 - accuracy: 0.6484 - val_loss: 0.0426 - val_accuracy: 0.9898
Epoch 2/10
51/51 [==============================] - 133s 3s/step - loss: 0.8605 - accuracy: 0.7632 - val_loss: 0.0535 - val_accuracy: 0.9900
Epoch 3/10
51/51 [==============================] - 128s 3s/step - loss: 0.6767 - accuracy: 0.8102 - val_loss: 0.0936 - val_accuracy: 0.9801
Epoch 4/10
51/51 [==============================] - 138s 3s/step - loss: 0.5692 - accuracy: 0.8326 - val_loss: 0.0457 - val_accuracy: 0.9880
Epoch 5/10
51/51 [==============================] - 128s 3s/step - loss: 0.5294 - accuracy: 0.8528 - val_loss: 0.0371 - val_accuracy: 0.9883
Epoch 6/10
51/51 [==============================] - 139s 3s/step - loss: 0.4762 - accuracy: 0.8606 - val_loss: 0.0981 - val_accuracy: 0.9747
Epoch 7/10
51/51 [==============================] - 135s 3s/step - loss: 0.3961 - accuracy: 0.8808 - val_loss: 0.0399 - val_accuracy: 0.9888
Epoch 8/10

In [139]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

17
Epoch 1/10
54/54 [==============================] - 162s 3s/step - loss: 1.3693 - accuracy: 0.6832 - val_loss: 0.0843 - val_accuracy: 0.9860
Epoch 2/10
54/54 [==============================] - 156s 3s/step - loss: 0.8028 - accuracy: 0.7777 - val_loss: 0.0552 - val_accuracy: 0.9829
Epoch 3/10
54/54 [==============================] - 168s 3s/step - loss: 0.6170 - accuracy: 0.8171 - val_loss: 0.0367 - val_accuracy: 0.9913
Epoch 4/10
54/54 [==============================] - 147s 3s/step - loss: 0.5431 - accuracy: 0.8421 - val_loss: 0.0535 - val_accuracy: 0.9872
Epoch 5/10
54/54 [==============================] - 162s 3s/step - loss: 0.4610 - accuracy: 0.8668 - val_loss: 0.0431 - val_accuracy: 0.9888
Epoch 6/10
54/54 [==============================] - 159s 3s/step - loss: 0.4059 - accuracy: 0.8753 - val_loss: 0.0434 - val_accuracy: 0.9913
Epoch 7/10
54/54 [==============================] - 160s 3s/step - loss: 0.3439 - accuracy: 0.8981 - val_loss: 0.0361 - val_accuracy: 0.9898
Epoch 8/10

In [140]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

18
Epoch 1/10
57/57 [==============================] - 177s 3s/step - loss: 1.2797 - accuracy: 0.6933 - val_loss: 0.0462 - val_accuracy: 0.9883
Epoch 2/10
57/57 [==============================] - 171s 3s/step - loss: 0.7243 - accuracy: 0.8062 - val_loss: 0.0535 - val_accuracy: 0.9852
Epoch 3/10
57/57 [==============================] - 174s 3s/step - loss: 0.6079 - accuracy: 0.8309 - val_loss: 0.0428 - val_accuracy: 0.9895
Epoch 4/10
57/57 [==============================] - 175s 3s/step - loss: 0.4984 - accuracy: 0.8627 - val_loss: 0.0383 - val_accuracy: 0.9916
Epoch 5/10
57/57 [==============================] - 176s 3s/step - loss: 0.4400 - accuracy: 0.8743 - val_loss: 0.0405 - val_accuracy: 0.9918
Epoch 6/10
57/57 [==============================] - 168s 3s/step - loss: 0.3947 - accuracy: 0.8824 - val_loss: 0.0547 - val_accuracy: 0.9872
Epoch 7/10
57/57 [==============================] - 173s 3s/step - loss: 0.3455 - accuracy: 0.8929 - val_loss: 0.0510 - val_accuracy: 0.9883
Epoch 8/10

In [141]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

19
Epoch 1/10
62/62 [==============================] - 192s 3s/step - loss: 1.1453 - accuracy: 0.7226 - val_loss: 0.1268 - val_accuracy: 0.9676
Epoch 2/10
62/62 [==============================] - 193s 3s/step - loss: 0.7500 - accuracy: 0.7983 - val_loss: 0.0279 - val_accuracy: 0.9934
Epoch 3/10
62/62 [==============================] - 185s 3s/step - loss: 0.5054 - accuracy: 0.8543 - val_loss: 0.0390 - val_accuracy: 0.9898
Epoch 4/10
62/62 [==============================] - 181s 3s/step - loss: 0.4174 - accuracy: 0.8748 - val_loss: 0.0967 - val_accuracy: 0.9727
Epoch 5/10
62/62 [==============================] - 173s 3s/step - loss: 0.4005 - accuracy: 0.8873 - val_loss: 0.0516 - val_accuracy: 0.9867
Epoch 6/10
62/62 [==============================] - 187s 3s/step - loss: 0.3562 - accuracy: 0.8899 - val_loss: 0.0380 - val_accuracy: 0.9911
Epoch 7/10
62/62 [==============================] - 185s 3s/step - loss: 0.3440 - accuracy: 0.9018 - val_loss: 0.0608 - val_accuracy: 0.9867
Epoch 8/10

In [142]:
#new_model_se_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_se_e1"
new_model_se_dir  = "D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_se_e2_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug

In [143]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [144]:
gc.collect()

526135

## Training guided by Random values

In [145]:
#save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/random_values.npy"
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)[len(x_train):] 
print(len(random_indexes))

3921


In [146]:
len(random_indexes)

3921

In [147]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_adversary_training)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_adversary_training)

In [148]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  190
190
1 :
0  ->  380
380
2 :
0  ->  570
570
3 :
0  ->  760
760
4 :
0  ->  950
950
5 :
0  ->  1140
1140
6 :
0  ->  1330
1330
7 :
0  ->  1520
1520
8 :
0  ->  1710
1710
9 :
0  ->  1900
1900
10 :
0  ->  2090
2090
11 :
0  ->  2280
2280
12 :
0  ->  2470
2470
13 :
0  ->  2660
2660
14 :
0  ->  2850
2850
15 :
0  ->  3040
3040
16 :
0  ->  3230
3230
17 :
0  ->  3420
3420
18 :
0  ->  3610
3610
19 :
Last
0  ->  3921
3921


In [149]:
len(image_sets_random)

20

In [150]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [151]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

0
Epoch 1/10
3/3 [==============================] - 19s 7s/step - loss: 1.7873 - accuracy: 0.6570 - val_loss: 0.1287 - val_accuracy: 0.9765
Epoch 2/10
3/3 [==============================] - 15s 7s/step - loss: 1.1078 - accuracy: 0.7095 - val_loss: 0.0382 - val_accuracy: 0.9900
Epoch 3/10
3/3 [==============================] - 15s 7s/step - loss: 0.7309 - accuracy: 0.7808 - val_loss: 0.0546 - val_accuracy: 0.9890
Epoch 4/10
3/3 [==============================] - 16s 7s/step - loss: 0.5909 - accuracy: 0.8729 - val_loss: 0.0507 - val_accuracy: 0.9880
Epoch 5/10
3/3 [==============================] - 16s 8s/step - loss: 0.4791 - accuracy: 0.8709 - val_loss: 0.0632 - val_accuracy: 0.9842
Epoch 6/10
3/3 [==============================] - 16s 8s/step - loss: 0.5978 - accuracy: 0.8469 - val_loss: 0.0437 - val_accuracy: 0.9893
Epoch 7/10
3/3 [==============================] - 16s 8s/step - loss: 0.4700 - accuracy: 0.8482 - val_loss: 0.0690 - val_accuracy: 0.9811
Epoch 8/10
3/3 [================

In [152]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

1
Epoch 1/10
6/6 [==============================] - 22s 4s/step - loss: 1.4954 - accuracy: 0.7225 - val_loss: 0.0676 - val_accuracy: 0.9847
Epoch 2/10
6/6 [==============================] - 18s 4s/step - loss: 0.9872 - accuracy: 0.7491 - val_loss: 0.0427 - val_accuracy: 0.9890
Epoch 3/10
6/6 [==============================] - 18s 4s/step - loss: 0.7422 - accuracy: 0.8073 - val_loss: 0.0344 - val_accuracy: 0.9906
Epoch 4/10
6/6 [==============================] - 19s 4s/step - loss: 0.8446 - accuracy: 0.7765 - val_loss: 0.0437 - val_accuracy: 0.9888
Epoch 5/10
6/6 [==============================] - 19s 4s/step - loss: 0.6754 - accuracy: 0.8381 - val_loss: 0.0438 - val_accuracy: 0.9900
Epoch 6/10
6/6 [==============================] - 20s 4s/step - loss: 0.5753 - accuracy: 0.8228 - val_loss: 0.0334 - val_accuracy: 0.9931
Epoch 7/10
6/6 [==============================] - 18s 4s/step - loss: 0.5072 - accuracy: 0.8583 - val_loss: 0.0590 - val_accuracy: 0.9883
Epoch 8/10
6/6 [================

In [153]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

2
Epoch 1/10
9/9 [==============================] - 33s 4s/step - loss: 1.8804 - accuracy: 0.6213 - val_loss: 0.0863 - val_accuracy: 0.9875
Epoch 2/10
9/9 [==============================] - 30s 4s/step - loss: 0.9204 - accuracy: 0.7621 - val_loss: 0.0489 - val_accuracy: 0.9900
Epoch 3/10
9/9 [==============================] - 29s 4s/step - loss: 0.7743 - accuracy: 0.7819 - val_loss: 0.0450 - val_accuracy: 0.9893
Epoch 4/10
9/9 [==============================] - 30s 4s/step - loss: 0.7071 - accuracy: 0.7998 - val_loss: 0.0498 - val_accuracy: 0.9885
Epoch 5/10
9/9 [==============================] - 29s 4s/step - loss: 0.5757 - accuracy: 0.8310 - val_loss: 0.0755 - val_accuracy: 0.9857
Epoch 6/10
9/9 [==============================] - 31s 4s/step - loss: 0.5620 - accuracy: 0.8320 - val_loss: 0.0470 - val_accuracy: 0.9908
Epoch 7/10
9/9 [==============================] - 31s 4s/step - loss: 0.5356 - accuracy: 0.8490 - val_loss: 0.0645 - val_accuracy: 0.9880
Epoch 8/10
9/9 [================

In [154]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

3
Epoch 1/10
12/12 [==============================] - 39s 3s/step - loss: 1.7010 - accuracy: 0.6788 - val_loss: 0.0527 - val_accuracy: 0.9875
Epoch 2/10
12/12 [==============================] - 34s 3s/step - loss: 0.9940 - accuracy: 0.7550 - val_loss: 0.0980 - val_accuracy: 0.9842
Epoch 3/10
12/12 [==============================] - 34s 3s/step - loss: 0.8691 - accuracy: 0.7592 - val_loss: 0.0418 - val_accuracy: 0.9913
Epoch 4/10
12/12 [==============================] - 44s 4s/step - loss: 0.6514 - accuracy: 0.8112 - val_loss: 0.0517 - val_accuracy: 0.9898
Epoch 5/10
12/12 [==============================] - 45s 4s/step - loss: 0.5727 - accuracy: 0.8596 - val_loss: 0.0310 - val_accuracy: 0.9923
Epoch 6/10
12/12 [==============================] - 38s 3s/step - loss: 0.4800 - accuracy: 0.8675 - val_loss: 0.0292 - val_accuracy: 0.9918
Epoch 7/10
12/12 [==============================] - 45s 4s/step - loss: 0.4260 - accuracy: 0.8681 - val_loss: 0.0466 - val_accuracy: 0.9921
Epoch 8/10
12/12 [

In [155]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

4
Epoch 1/10
15/15 [==============================] - 50s 3s/step - loss: 1.4968 - accuracy: 0.6781 - val_loss: 0.0573 - val_accuracy: 0.9911
Epoch 2/10
15/15 [==============================] - 41s 3s/step - loss: 0.8258 - accuracy: 0.7779 - val_loss: 0.1090 - val_accuracy: 0.9737
Epoch 3/10
15/15 [==============================] - 46s 3s/step - loss: 0.8375 - accuracy: 0.7762 - val_loss: 0.0457 - val_accuracy: 0.9895
Epoch 4/10
15/15 [==============================] - 42s 3s/step - loss: 0.6717 - accuracy: 0.8200 - val_loss: 0.0406 - val_accuracy: 0.9921
Epoch 5/10
15/15 [==============================] - 43s 3s/step - loss: 0.5685 - accuracy: 0.8387 - val_loss: 0.0964 - val_accuracy: 0.9852
Epoch 6/10
15/15 [==============================] - 43s 3s/step - loss: 0.4319 - accuracy: 0.8833 - val_loss: 0.0508 - val_accuracy: 0.9893
Epoch 7/10
15/15 [==============================] - 47s 3s/step - loss: 0.4854 - accuracy: 0.8541 - val_loss: 0.0554 - val_accuracy: 0.9878
Epoch 8/10
15/15 [

In [156]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

5
Epoch 1/10
18/18 [==============================] - 52s 3s/step - loss: 1.4664 - accuracy: 0.6781 - val_loss: 0.0589 - val_accuracy: 0.9878
Epoch 2/10
18/18 [==============================] - 50s 3s/step - loss: 0.9198 - accuracy: 0.7672 - val_loss: 0.0357 - val_accuracy: 0.9913
Epoch 3/10
18/18 [==============================] - 49s 3s/step - loss: 0.7109 - accuracy: 0.7995 - val_loss: 0.0385 - val_accuracy: 0.9926
Epoch 4/10
18/18 [==============================] - 62s 4s/step - loss: 0.6440 - accuracy: 0.8102 - val_loss: 0.0512 - val_accuracy: 0.9890
Epoch 5/10
18/18 [==============================] - 53s 3s/step - loss: 0.5482 - accuracy: 0.8338 - val_loss: 0.0544 - val_accuracy: 0.9890
Epoch 6/10
18/18 [==============================] - 51s 3s/step - loss: 0.5241 - accuracy: 0.8538 - val_loss: 0.0505 - val_accuracy: 0.9883
Epoch 7/10
18/18 [==============================] - 49s 3s/step - loss: 0.4843 - accuracy: 0.8683 - val_loss: 0.0347 - val_accuracy: 0.9906
Epoch 8/10
18/18 [

In [157]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

6
Epoch 1/10
21/21 [==============================] - 69s 3s/step - loss: 1.5353 - accuracy: 0.6851 - val_loss: 0.0315 - val_accuracy: 0.9923
Epoch 2/10
21/21 [==============================] - 66s 3s/step - loss: 0.8877 - accuracy: 0.7624 - val_loss: 0.0385 - val_accuracy: 0.9895
Epoch 3/10
21/21 [==============================] - 56s 3s/step - loss: 0.7293 - accuracy: 0.7959 - val_loss: 0.0388 - val_accuracy: 0.9911
Epoch 4/10
21/21 [==============================] - 70s 3s/step - loss: 0.6110 - accuracy: 0.8367 - val_loss: 0.0315 - val_accuracy: 0.9926
Epoch 5/10
21/21 [==============================] - 59s 3s/step - loss: 0.5762 - accuracy: 0.8519 - val_loss: 0.0335 - val_accuracy: 0.9900
Epoch 6/10
21/21 [==============================] - 61s 3s/step - loss: 0.5027 - accuracy: 0.8519 - val_loss: 0.0573 - val_accuracy: 0.9893
Epoch 7/10
21/21 [==============================] - 55s 3s/step - loss: 0.4913 - accuracy: 0.8650 - val_loss: 0.0395 - val_accuracy: 0.9888
Epoch 8/10
21/21 [

In [158]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

7
Epoch 1/10
24/24 [==============================] - 79s 3s/step - loss: 1.3486 - accuracy: 0.6856 - val_loss: 0.0683 - val_accuracy: 0.9875
Epoch 2/10
24/24 [==============================] - 77s 3s/step - loss: 0.8748 - accuracy: 0.7737 - val_loss: 0.0372 - val_accuracy: 0.9906
Epoch 3/10
24/24 [==============================] - 86s 4s/step - loss: 0.7650 - accuracy: 0.7929 - val_loss: 0.0569 - val_accuracy: 0.9870
Epoch 4/10
24/24 [==============================] - 76s 3s/step - loss: 0.6641 - accuracy: 0.8287 - val_loss: 0.0383 - val_accuracy: 0.9903
Epoch 5/10
24/24 [==============================] - 77s 3s/step - loss: 0.5947 - accuracy: 0.8301 - val_loss: 0.0462 - val_accuracy: 0.9895
Epoch 6/10
24/24 [==============================] - 77s 3s/step - loss: 0.5391 - accuracy: 0.8429 - val_loss: 0.0387 - val_accuracy: 0.9918
Epoch 7/10
24/24 [==============================] - 77s 3s/step - loss: 0.4157 - accuracy: 0.8707 - val_loss: 0.0476 - val_accuracy: 0.9890
Epoch 8/10
24/24 [

In [159]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

8
Epoch 1/10
27/27 [==============================] - 100s 4s/step - loss: 1.4544 - accuracy: 0.6768 - val_loss: 0.0452 - val_accuracy: 0.9893
Epoch 2/10
27/27 [==============================] - 105s 4s/step - loss: 0.8553 - accuracy: 0.7800 - val_loss: 0.0413 - val_accuracy: 0.9906
Epoch 3/10
27/27 [==============================] - 86s 3s/step - loss: 0.7153 - accuracy: 0.8047 - val_loss: 0.0535 - val_accuracy: 0.9872
Epoch 4/10
27/27 [==============================] - 87s 3s/step - loss: 0.5906 - accuracy: 0.8374 - val_loss: 0.0329 - val_accuracy: 0.9906
Epoch 5/10
27/27 [==============================] - 101s 4s/step - loss: 0.4859 - accuracy: 0.8626 - val_loss: 0.0321 - val_accuracy: 0.9921
Epoch 6/10
27/27 [==============================] - 94s 4s/step - loss: 0.4831 - accuracy: 0.8530 - val_loss: 0.0556 - val_accuracy: 0.9883
Epoch 7/10
27/27 [==============================] - 99s 4s/step - loss: 0.5145 - accuracy: 0.8508 - val_loss: 0.0403 - val_accuracy: 0.9893
Epoch 8/10
27/2

In [160]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

9
Epoch 1/10
30/30 [==============================] - 104s 3s/step - loss: 1.4610 - accuracy: 0.6696 - val_loss: 0.0418 - val_accuracy: 0.9900
Epoch 2/10
30/30 [==============================] - 108s 4s/step - loss: 0.8537 - accuracy: 0.7778 - val_loss: 0.0546 - val_accuracy: 0.9870
Epoch 3/10
30/30 [==============================] - 108s 4s/step - loss: 0.7183 - accuracy: 0.8020 - val_loss: 0.0516 - val_accuracy: 0.9875
Epoch 4/10
30/30 [==============================] - 113s 4s/step - loss: 0.6087 - accuracy: 0.8250 - val_loss: 0.0420 - val_accuracy: 0.9903
Epoch 5/10
30/30 [==============================] - 94s 3s/step - loss: 0.5768 - accuracy: 0.8195 - val_loss: 0.0838 - val_accuracy: 0.9811
Epoch 6/10
30/30 [==============================] - 104s 4s/step - loss: 0.4342 - accuracy: 0.8701 - val_loss: 0.0502 - val_accuracy: 0.9878
Epoch 7/10
30/30 [==============================] - 94s 3s/step - loss: 0.4352 - accuracy: 0.8663 - val_loss: 0.0555 - val_accuracy: 0.9860
Epoch 8/10
30

In [161]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

10
Epoch 1/10
33/33 [==============================] - 103s 3s/step - loss: 1.5028 - accuracy: 0.6585 - val_loss: 0.0821 - val_accuracy: 0.9809
Epoch 2/10
33/33 [==============================] - 101s 3s/step - loss: 0.8764 - accuracy: 0.7710 - val_loss: 0.0398 - val_accuracy: 0.9908
Epoch 3/10
33/33 [==============================] - 110s 3s/step - loss: 0.6824 - accuracy: 0.8049 - val_loss: 0.0359 - val_accuracy: 0.9918
Epoch 4/10
33/33 [==============================] - 103s 3s/step - loss: 0.5424 - accuracy: 0.8426 - val_loss: 0.0438 - val_accuracy: 0.9903
Epoch 5/10
33/33 [==============================] - 101s 3s/step - loss: 0.6064 - accuracy: 0.8292 - val_loss: 0.0418 - val_accuracy: 0.9878
Epoch 6/10
33/33 [==============================] - 110s 3s/step - loss: 0.3996 - accuracy: 0.8900 - val_loss: 0.0355 - val_accuracy: 0.9918
Epoch 7/10
33/33 [==============================] - 95s 3s/step - loss: 0.3793 - accuracy: 0.8809 - val_loss: 0.0454 - val_accuracy: 0.9906
Epoch 8/10


In [162]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

11
Epoch 1/10
36/36 [==============================] - 117s 3s/step - loss: 1.2421 - accuracy: 0.7216 - val_loss: 0.0798 - val_accuracy: 0.9860
Epoch 2/10
36/36 [==============================] - 106s 3s/step - loss: 0.7620 - accuracy: 0.7960 - val_loss: 0.0300 - val_accuracy: 0.9929
Epoch 3/10
36/36 [==============================] - 107s 3s/step - loss: 0.6700 - accuracy: 0.8310 - val_loss: 0.0350 - val_accuracy: 0.9908
Epoch 4/10
36/36 [==============================] - 113s 3s/step - loss: 0.5790 - accuracy: 0.8311 - val_loss: 0.0413 - val_accuracy: 0.9883
Epoch 5/10
36/36 [==============================] - 117s 3s/step - loss: 0.4873 - accuracy: 0.8519 - val_loss: 0.0382 - val_accuracy: 0.9900
Epoch 6/10
36/36 [==============================] - 108s 3s/step - loss: 0.4461 - accuracy: 0.8709 - val_loss: 0.0646 - val_accuracy: 0.9829
Epoch 7/10
36/36 [==============================] - 120s 3s/step - loss: 0.3974 - accuracy: 0.8876 - val_loss: 0.0352 - val_accuracy: 0.9908
Epoch 8/10

In [163]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

12
Epoch 1/10
39/39 [==============================] - 141s 4s/step - loss: 1.2378 - accuracy: 0.7134 - val_loss: 0.0315 - val_accuracy: 0.9921
Epoch 2/10
39/39 [==============================] - 133s 3s/step - loss: 0.7158 - accuracy: 0.8095 - val_loss: 0.0314 - val_accuracy: 0.9931
Epoch 3/10
39/39 [==============================] - 124s 3s/step - loss: 0.6358 - accuracy: 0.8354 - val_loss: 0.0468 - val_accuracy: 0.9883
Epoch 4/10
39/39 [==============================] - 137s 4s/step - loss: 0.5356 - accuracy: 0.8453 - val_loss: 0.0507 - val_accuracy: 0.9875
Epoch 5/10
39/39 [==============================] - 124s 3s/step - loss: 0.4399 - accuracy: 0.8663 - val_loss: 0.0552 - val_accuracy: 0.9862
Epoch 6/10
39/39 [==============================] - 136s 4s/step - loss: 0.4381 - accuracy: 0.8648 - val_loss: 0.0373 - val_accuracy: 0.9903
Epoch 7/10
39/39 [==============================] - 126s 3s/step - loss: 0.3931 - accuracy: 0.8841 - val_loss: 0.0347 - val_accuracy: 0.9903
Epoch 8/10

In [164]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

13
Epoch 1/10
42/42 [==============================] - 156s 4s/step - loss: 1.3216 - accuracy: 0.6954 - val_loss: 0.0486 - val_accuracy: 0.9898
Epoch 2/10
42/42 [==============================] - 141s 3s/step - loss: 0.7532 - accuracy: 0.7953 - val_loss: 0.0471 - val_accuracy: 0.9875
Epoch 3/10
42/42 [==============================] - 155s 4s/step - loss: 0.6090 - accuracy: 0.8231 - val_loss: 0.0434 - val_accuracy: 0.9906
Epoch 4/10
42/42 [==============================] - 152s 4s/step - loss: 0.5458 - accuracy: 0.8403 - val_loss: 0.0457 - val_accuracy: 0.9875
Epoch 5/10
42/42 [==============================] - 147s 4s/step - loss: 0.4756 - accuracy: 0.8617 - val_loss: 0.0479 - val_accuracy: 0.9898
Epoch 6/10
42/42 [==============================] - 134s 3s/step - loss: 0.4397 - accuracy: 0.8697 - val_loss: 0.0366 - val_accuracy: 0.9923
Epoch 7/10
42/42 [==============================] - 151s 4s/step - loss: 0.3589 - accuracy: 0.8954 - val_loss: 0.0713 - val_accuracy: 0.9832
Epoch 8/10

In [165]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

14
Epoch 1/10
45/45 [==============================] - 166s 4s/step - loss: 1.3425 - accuracy: 0.6873 - val_loss: 0.0397 - val_accuracy: 0.9890
Epoch 2/10
45/45 [==============================] - 175s 4s/step - loss: 0.7359 - accuracy: 0.7953 - val_loss: 0.0498 - val_accuracy: 0.9872
Epoch 3/10
45/45 [==============================] - 179s 4s/step - loss: 0.5998 - accuracy: 0.8265 - val_loss: 0.0575 - val_accuracy: 0.9857
Epoch 4/10
45/45 [==============================] - 174s 4s/step - loss: 0.5260 - accuracy: 0.8572 - val_loss: 0.0654 - val_accuracy: 0.9819
Epoch 5/10
45/45 [==============================] - 173s 4s/step - loss: 0.5005 - accuracy: 0.8608 - val_loss: 0.0399 - val_accuracy: 0.9893
Epoch 6/10
45/45 [==============================] - 178s 4s/step - loss: 0.4146 - accuracy: 0.8789 - val_loss: 0.0458 - val_accuracy: 0.9883
Epoch 7/10
45/45 [==============================] - 162s 4s/step - loss: 0.3618 - accuracy: 0.8933 - val_loss: 0.0681 - val_accuracy: 0.9829
Epoch 8/10

In [166]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

15
Epoch 1/10
48/48 [==============================] - 200s 4s/step - loss: 1.2837 - accuracy: 0.7114 - val_loss: 0.0426 - val_accuracy: 0.9908
Epoch 2/10
48/48 [==============================] - 196s 4s/step - loss: 0.6957 - accuracy: 0.8155 - val_loss: 0.0429 - val_accuracy: 0.9903
Epoch 3/10
48/48 [==============================] - 197s 4s/step - loss: 0.5942 - accuracy: 0.8299 - val_loss: 0.0430 - val_accuracy: 0.9885
Epoch 4/10
48/48 [==============================] - 197s 4s/step - loss: 0.4999 - accuracy: 0.8605 - val_loss: 0.0360 - val_accuracy: 0.9926
Epoch 5/10
48/48 [==============================] - 204s 4s/step - loss: 0.4397 - accuracy: 0.8701 - val_loss: 0.0440 - val_accuracy: 0.9880
Epoch 6/10
48/48 [==============================] - 193s 4s/step - loss: 0.3512 - accuracy: 0.8998 - val_loss: 0.0361 - val_accuracy: 0.9895
Epoch 7/10
48/48 [==============================] - 202s 4s/step - loss: 0.3258 - accuracy: 0.8994 - val_loss: 0.0357 - val_accuracy: 0.9908
Epoch 8/10

In [167]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

16
Epoch 1/10
51/51 [==============================] - 253s 5s/step - loss: 1.2218 - accuracy: 0.7231 - val_loss: 0.0360 - val_accuracy: 0.9911
Epoch 2/10
51/51 [==============================] - 244s 5s/step - loss: 0.7406 - accuracy: 0.8038 - val_loss: 0.0321 - val_accuracy: 0.9934
Epoch 3/10
51/51 [==============================] - 240s 5s/step - loss: 0.5672 - accuracy: 0.8392 - val_loss: 0.0349 - val_accuracy: 0.9898
Epoch 4/10
51/51 [==============================] - 243s 5s/step - loss: 0.4636 - accuracy: 0.8670 - val_loss: 0.0576 - val_accuracy: 0.9849
Epoch 5/10
51/51 [==============================] - 214s 4s/step - loss: 0.4908 - accuracy: 0.8543 - val_loss: 0.0303 - val_accuracy: 0.9906
Epoch 6/10
51/51 [==============================] - 202s 4s/step - loss: 0.3679 - accuracy: 0.8934 - val_loss: 0.0429 - val_accuracy: 0.9895
Epoch 7/10
51/51 [==============================] - 207s 4s/step - loss: 0.3502 - accuracy: 0.8923 - val_loss: 0.0918 - val_accuracy: 0.9752
Epoch 8/10

In [168]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

17
Epoch 1/10
54/54 [==============================] - 207s 4s/step - loss: 1.2936 - accuracy: 0.7115 - val_loss: 0.0492 - val_accuracy: 0.9872
Epoch 2/10
54/54 [==============================] - 229s 4s/step - loss: 0.7261 - accuracy: 0.7958 - val_loss: 0.0466 - val_accuracy: 0.9918
Epoch 3/10
54/54 [==============================] - 216s 4s/step - loss: 0.6014 - accuracy: 0.8385 - val_loss: 0.0339 - val_accuracy: 0.9906
Epoch 4/10
54/54 [==============================] - 211s 4s/step - loss: 0.4856 - accuracy: 0.8586 - val_loss: 0.0357 - val_accuracy: 0.9908
Epoch 5/10
54/54 [==============================] - 180s 3s/step - loss: 0.4098 - accuracy: 0.8774 - val_loss: 0.1245 - val_accuracy: 0.9717
Epoch 6/10
54/54 [==============================] - 198s 4s/step - loss: 0.4062 - accuracy: 0.8827 - val_loss: 0.0400 - val_accuracy: 0.9895
Epoch 7/10
54/54 [==============================] - 195s 4s/step - loss: 0.3456 - accuracy: 0.9011 - val_loss: 0.0408 - val_accuracy: 0.9888
Epoch 8/10

In [169]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

18
Epoch 1/10
57/57 [==============================] - 239s 4s/step - loss: 1.1980 - accuracy: 0.7192 - val_loss: 0.0622 - val_accuracy: 0.9883
Epoch 2/10
57/57 [==============================] - 230s 4s/step - loss: 0.6495 - accuracy: 0.8153 - val_loss: 0.0910 - val_accuracy: 0.9862
Epoch 3/10
57/57 [==============================] - 234s 4s/step - loss: 0.5894 - accuracy: 0.8370 - val_loss: 0.0528 - val_accuracy: 0.9862
Epoch 4/10
57/57 [==============================] - 230s 4s/step - loss: 0.4633 - accuracy: 0.8594 - val_loss: 0.0423 - val_accuracy: 0.9893
Epoch 5/10
57/57 [==============================] - 237s 4s/step - loss: 0.4159 - accuracy: 0.8766 - val_loss: 0.0857 - val_accuracy: 0.9849
Epoch 6/10
57/57 [==============================] - 242s 4s/step - loss: 0.3943 - accuracy: 0.8866 - val_loss: 0.0453 - val_accuracy: 0.9918
Epoch 7/10
57/57 [==============================] - 242s 4s/step - loss: 0.3667 - accuracy: 0.8909 - val_loss: 0.0476 - val_accuracy: 0.9880
Epoch 8/10

In [170]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

19
Epoch 1/10
62/62 [==============================] - 259s 4s/step - loss: 1.0905 - accuracy: 0.7412 - val_loss: 0.0390 - val_accuracy: 0.9908
Epoch 2/10
62/62 [==============================] - 244s 4s/step - loss: 0.7167 - accuracy: 0.7990 - val_loss: 0.0426 - val_accuracy: 0.9875
Epoch 3/10
62/62 [==============================] - 254s 4s/step - loss: 0.5599 - accuracy: 0.8360 - val_loss: 0.1725 - val_accuracy: 0.9704
Epoch 4/10
62/62 [==============================] - 249s 4s/step - loss: 0.4911 - accuracy: 0.8565 - val_loss: 0.1311 - val_accuracy: 0.9707
Epoch 5/10
62/62 [==============================] - 255s 4s/step - loss: 0.3936 - accuracy: 0.8754 - val_loss: 0.0652 - val_accuracy: 0.9798
Epoch 6/10
62/62 [==============================] - 258s 4s/step - loss: 0.3561 - accuracy: 0.8928 - val_loss: 0.0595 - val_accuracy: 0.9862
Epoch 7/10
62/62 [==============================] - 255s 4s/step - loss: 0.3192 - accuracy: 0.9099 - val_loss: 0.0349 - val_accuracy: 0.9921
Epoch 8/10

In [171]:
#new_model_random_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_random_e1"
new_model_random_dir  = "D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_random_e2_7\assets
Model has been saved
INFO:tensorflow:

In [172]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [173]:
import gc

In [174]:
gc.collect()

508036

## Training guided by NC

In [20]:
# NC
nc_values = []
for i in range(1,13):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    #save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/gtsrb_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)
    
nc_values = nc_values[len(x_train):] 
print(len(nc_values))

3921


In [21]:
nc_values.shape

(3921,)

In [22]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_adversary_training)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_adversary_training)


In [23]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  190
190
1 :
0  ->  380
380
2 :
0  ->  570
570
3 :
0  ->  760
760
4 :
0  ->  950
950
5 :
0  ->  1140
1140
6 :
0  ->  1330
1330
7 :
0  ->  1520
1520
8 :
0  ->  1710
1710
9 :
0  ->  1900
1900
10 :
0  ->  2090
2090
11 :
0  ->  2280
2280
12 :
0  ->  2470
2470
13 :
0  ->  2660
2660
14 :
0  ->  2850
2850
15 :
0  ->  3040
3040
16 :
0  ->  3230
3230
17 :
0  ->  3420
3420
18 :
0  ->  3610
3610
19 :
Last
0  ->  3921
3921


In [24]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [25]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

0
Epoch 1/10
3/3 [==============================] - 30s 3s/step - loss: 0.8557 - accuracy: 0.8658 - val_loss: 0.7573 - val_accuracy: 0.8551
Epoch 2/10
3/3 [==============================] - 6s 3s/step - loss: 0.2570 - accuracy: 0.9418 - val_loss: 0.0808 - val_accuracy: 0.9758
Epoch 3/10
3/3 [==============================] - 6s 3s/step - loss: 0.1249 - accuracy: 0.9437 - val_loss: 0.0648 - val_accuracy: 0.9806
Epoch 4/10
3/3 [==============================] - 7s 3s/step - loss: 0.1409 - accuracy: 0.9607 - val_loss: 0.0547 - val_accuracy: 0.9862
Epoch 5/10
3/3 [==============================] - 7s 3s/step - loss: 0.1607 - accuracy: 0.9568 - val_loss: 0.0750 - val_accuracy: 0.9778
Epoch 6/10
3/3 [==============================] - 7s 3s/step - loss: 0.1337 - accuracy: 0.9620 - val_loss: 0.0723 - val_accuracy: 0.9793
Epoch 7/10
3/3 [==============================] - 7s 3s/step - loss: 0.1042 - accuracy: 0.9705 - val_loss: 0.0655 - val_accuracy: 0.9816
Epoch 8/10
3/3 [======================

In [26]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

1
Epoch 1/10
6/6 [==============================] - 11s 2s/step - loss: 0.9257 - accuracy: 0.8426 - val_loss: 0.1675 - val_accuracy: 0.9490
Epoch 2/10
6/6 [==============================] - 8s 2s/step - loss: 0.3750 - accuracy: 0.9119 - val_loss: 0.0954 - val_accuracy: 0.9735
Epoch 3/10
6/6 [==============================] - 8s 2s/step - loss: 0.2932 - accuracy: 0.9217 - val_loss: 0.0755 - val_accuracy: 0.9786
Epoch 4/10
6/6 [==============================] - 8s 2s/step - loss: 0.2076 - accuracy: 0.9263 - val_loss: 0.0805 - val_accuracy: 0.9775
Epoch 5/10
6/6 [==============================] - 8s 2s/step - loss: 0.2305 - accuracy: 0.9385 - val_loss: 0.0846 - val_accuracy: 0.9758
Epoch 6/10
6/6 [==============================] - 8s 2s/step - loss: 0.2044 - accuracy: 0.9536 - val_loss: 0.1138 - val_accuracy: 0.9671
Epoch 7/10
6/6 [==============================] - 8s 2s/step - loss: 0.1582 - accuracy: 0.9602 - val_loss: 0.1120 - val_accuracy: 0.9727
Epoch 8/10
6/6 [======================

In [27]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

2
Epoch 1/10
9/9 [==============================] - 12s 1s/step - loss: 1.0846 - accuracy: 0.8114 - val_loss: 0.0510 - val_accuracy: 0.9862
Epoch 2/10
9/9 [==============================] - 10s 1s/step - loss: 0.4430 - accuracy: 0.8810 - val_loss: 0.0430 - val_accuracy: 0.9893
Epoch 3/10
9/9 [==============================] - 10s 1s/step - loss: 0.4482 - accuracy: 0.8775 - val_loss: 0.1139 - val_accuracy: 0.9671
Epoch 4/10
9/9 [==============================] - 10s 1s/step - loss: 0.4815 - accuracy: 0.8692 - val_loss: 0.0915 - val_accuracy: 0.9727
Epoch 5/10
9/9 [==============================] - 10s 1s/step - loss: 0.3261 - accuracy: 0.8990 - val_loss: 0.0580 - val_accuracy: 0.9857
Epoch 6/10
9/9 [==============================] - 10s 1s/step - loss: 0.3667 - accuracy: 0.9016 - val_loss: 0.0460 - val_accuracy: 0.9883
Epoch 7/10
9/9 [==============================] - 10s 1s/step - loss: 0.3375 - accuracy: 0.9087 - val_loss: 0.0971 - val_accuracy: 0.9724
Epoch 8/10
9/9 [================

In [28]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

3
Epoch 1/10
12/12 [==============================] - 13s 936ms/step - loss: 0.9662 - accuracy: 0.8354 - val_loss: 0.0538 - val_accuracy: 0.9852
Epoch 2/10
12/12 [==============================] - 11s 943ms/step - loss: 0.4343 - accuracy: 0.8782 - val_loss: 0.1481 - val_accuracy: 0.9763
Epoch 3/10
12/12 [==============================] - 11s 940ms/step - loss: 0.4198 - accuracy: 0.8735 - val_loss: 0.0461 - val_accuracy: 0.9872
Epoch 4/10
12/12 [==============================] - 11s 940ms/step - loss: 0.3984 - accuracy: 0.8982 - val_loss: 0.0510 - val_accuracy: 0.9872
Epoch 5/10
12/12 [==============================] - 11s 945ms/step - loss: 0.3373 - accuracy: 0.8996 - val_loss: 0.0621 - val_accuracy: 0.9816
Epoch 6/10
12/12 [==============================] - 11s 941ms/step - loss: 0.2764 - accuracy: 0.9178 - val_loss: 0.0515 - val_accuracy: 0.9842
Epoch 7/10
12/12 [==============================] - 11s 938ms/step - loss: 0.2533 - accuracy: 0.9289 - val_loss: 0.0772 - val_accuracy: 0.97

In [29]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

4
Epoch 1/10
15/15 [==============================] - 14s 816ms/step - loss: 1.2306 - accuracy: 0.7939 - val_loss: 0.0306 - val_accuracy: 0.9921
Epoch 2/10
15/15 [==============================] - 12s 826ms/step - loss: 0.4725 - accuracy: 0.8811 - val_loss: 0.0567 - val_accuracy: 0.9867
Epoch 3/10
15/15 [==============================] - 13s 877ms/step - loss: 0.3586 - accuracy: 0.8885 - val_loss: 0.0460 - val_accuracy: 0.9878
Epoch 4/10
15/15 [==============================] - 12s 817ms/step - loss: 0.3577 - accuracy: 0.8950 - val_loss: 0.0584 - val_accuracy: 0.9862
Epoch 5/10
15/15 [==============================] - 12s 807ms/step - loss: 0.3430 - accuracy: 0.8979 - val_loss: 0.0749 - val_accuracy: 0.9796
Epoch 6/10
15/15 [==============================] - 12s 806ms/step - loss: 0.3214 - accuracy: 0.9085 - val_loss: 0.1549 - val_accuracy: 0.9533
Epoch 7/10
15/15 [==============================] - 12s 809ms/step - loss: 0.3010 - accuracy: 0.9002 - val_loss: 0.0730 - val_accuracy: 0.98

In [30]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

5
Epoch 1/10
18/18 [==============================] - 15s 747ms/step - loss: 0.8775 - accuracy: 0.8417 - val_loss: 0.0443 - val_accuracy: 0.9888
Epoch 2/10
18/18 [==============================] - 13s 745ms/step - loss: 0.4652 - accuracy: 0.8701 - val_loss: 0.0735 - val_accuracy: 0.9804
Epoch 3/10
18/18 [==============================] - 13s 740ms/step - loss: 0.4104 - accuracy: 0.8839 - val_loss: 0.1083 - val_accuracy: 0.9696
Epoch 4/10
18/18 [==============================] - 13s 736ms/step - loss: 0.3286 - accuracy: 0.8909 - val_loss: 0.0572 - val_accuracy: 0.9839
Epoch 5/10
18/18 [==============================] - 13s 739ms/step - loss: 0.3456 - accuracy: 0.9145 - val_loss: 0.0846 - val_accuracy: 0.9750
Epoch 6/10
18/18 [==============================] - 13s 741ms/step - loss: 0.3359 - accuracy: 0.9000 - val_loss: 0.1081 - val_accuracy: 0.9681
Epoch 7/10
18/18 [==============================] - 13s 743ms/step - loss: 0.2270 - accuracy: 0.9303 - val_loss: 0.1089 - val_accuracy: 0.97

In [31]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

6
Epoch 1/10
21/21 [==============================] - 17s 691ms/step - loss: 0.8150 - accuracy: 0.8271 - val_loss: 0.0666 - val_accuracy: 0.9875
Epoch 2/10
21/21 [==============================] - 14s 693ms/step - loss: 0.4695 - accuracy: 0.8692 - val_loss: 0.0840 - val_accuracy: 0.9847
Epoch 3/10
21/21 [==============================] - 14s 691ms/step - loss: 0.3846 - accuracy: 0.8849 - val_loss: 0.0806 - val_accuracy: 0.9806
Epoch 4/10
21/21 [==============================] - 14s 692ms/step - loss: 0.3149 - accuracy: 0.8986 - val_loss: 0.0673 - val_accuracy: 0.9847
Epoch 5/10
21/21 [==============================] - 14s 690ms/step - loss: 0.2972 - accuracy: 0.9186 - val_loss: 0.0622 - val_accuracy: 0.9867
Epoch 6/10
21/21 [==============================] - 14s 690ms/step - loss: 0.2926 - accuracy: 0.9130 - val_loss: 0.0913 - val_accuracy: 0.9798
Epoch 7/10
21/21 [==============================] - 14s 692ms/step - loss: 0.2213 - accuracy: 0.9336 - val_loss: 0.2387 - val_accuracy: 0.96

In [32]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

7
Epoch 1/10
24/24 [==============================] - 18s 660ms/step - loss: 0.9942 - accuracy: 0.8066 - val_loss: 0.0526 - val_accuracy: 0.9865
Epoch 2/10
24/24 [==============================] - 16s 658ms/step - loss: 0.3931 - accuracy: 0.8877 - val_loss: 0.0939 - val_accuracy: 0.9727
Epoch 3/10
24/24 [==============================] - 16s 657ms/step - loss: 0.4065 - accuracy: 0.8883 - val_loss: 0.0881 - val_accuracy: 0.9778
Epoch 4/10
24/24 [==============================] - 16s 659ms/step - loss: 0.3018 - accuracy: 0.9037 - val_loss: 0.0771 - val_accuracy: 0.9796
Epoch 5/10
24/24 [==============================] - 16s 659ms/step - loss: 0.3365 - accuracy: 0.9062 - val_loss: 0.0567 - val_accuracy: 0.9872
Epoch 6/10
24/24 [==============================] - 16s 662ms/step - loss: 0.2904 - accuracy: 0.9154 - val_loss: 0.0772 - val_accuracy: 0.9801
Epoch 7/10
24/24 [==============================] - 15s 654ms/step - loss: 0.2585 - accuracy: 0.9231 - val_loss: 0.1648 - val_accuracy: 0.95

In [33]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

8
Epoch 1/10
27/27 [==============================] - 19s 632ms/step - loss: 0.9624 - accuracy: 0.8000 - val_loss: 0.2758 - val_accuracy: 0.9661
Epoch 2/10
27/27 [==============================] - 17s 629ms/step - loss: 0.5022 - accuracy: 0.8657 - val_loss: 0.0385 - val_accuracy: 0.9911
Epoch 3/10
27/27 [==============================] - 17s 643ms/step - loss: 0.3577 - accuracy: 0.8960 - val_loss: 0.0501 - val_accuracy: 0.9857
Epoch 4/10
27/27 [==============================] - 17s 627ms/step - loss: 0.3107 - accuracy: 0.9075 - val_loss: 0.0726 - val_accuracy: 0.9834
Epoch 5/10
27/27 [==============================] - 17s 628ms/step - loss: 0.2683 - accuracy: 0.9285 - val_loss: 0.1397 - val_accuracy: 0.9663
Epoch 6/10
27/27 [==============================] - 17s 627ms/step - loss: 0.2728 - accuracy: 0.9230 - val_loss: 0.1011 - val_accuracy: 0.9742
Epoch 7/10
27/27 [==============================] - 17s 627ms/step - loss: 0.2668 - accuracy: 0.9280 - val_loss: 0.0969 - val_accuracy: 0.97

In [34]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

9
Epoch 1/10
30/30 [==============================] - 21s 606ms/step - loss: 0.8821 - accuracy: 0.8073 - val_loss: 0.0395 - val_accuracy: 0.9906
Epoch 2/10
30/30 [==============================] - 18s 609ms/step - loss: 0.4437 - accuracy: 0.8859 - val_loss: 0.0454 - val_accuracy: 0.9898
Epoch 3/10
30/30 [==============================] - 18s 607ms/step - loss: 0.3580 - accuracy: 0.8991 - val_loss: 0.0375 - val_accuracy: 0.9908
Epoch 4/10
30/30 [==============================] - 18s 607ms/step - loss: 0.3387 - accuracy: 0.9022 - val_loss: 0.0782 - val_accuracy: 0.9755
Epoch 5/10
30/30 [==============================] - 18s 609ms/step - loss: 0.3582 - accuracy: 0.8945 - val_loss: 0.0859 - val_accuracy: 0.9722
Epoch 6/10
30/30 [==============================] - 18s 608ms/step - loss: 0.2597 - accuracy: 0.9263 - val_loss: 0.2315 - val_accuracy: 0.9365
Epoch 7/10
30/30 [==============================] - 18s 611ms/step - loss: 0.2814 - accuracy: 0.9191 - val_loss: 0.1425 - val_accuracy: 0.96

In [35]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

10
Epoch 1/10
33/33 [==============================] - 22s 588ms/step - loss: 0.8370 - accuracy: 0.8387 - val_loss: 0.0928 - val_accuracy: 0.9781
Epoch 2/10
33/33 [==============================] - 20s 597ms/step - loss: 0.4487 - accuracy: 0.8798 - val_loss: 0.0699 - val_accuracy: 0.9875
Epoch 3/10
33/33 [==============================] - 19s 589ms/step - loss: 0.3788 - accuracy: 0.8987 - val_loss: 0.1368 - val_accuracy: 0.9747
Epoch 4/10
33/33 [==============================] - 19s 586ms/step - loss: 0.3609 - accuracy: 0.9011 - val_loss: 0.0848 - val_accuracy: 0.9760
Epoch 5/10
33/33 [==============================] - 19s 586ms/step - loss: 0.3012 - accuracy: 0.9070 - val_loss: 0.1017 - val_accuracy: 0.9714
Epoch 6/10
33/33 [==============================] - 19s 584ms/step - loss: 0.2383 - accuracy: 0.9311 - val_loss: 0.1253 - val_accuracy: 0.9656
Epoch 7/10
33/33 [==============================] - 24s 750ms/step - loss: 0.2571 - accuracy: 0.9256 - val_loss: 0.1804 - val_accuracy: 0.9

In [36]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

11
Epoch 1/10
36/36 [==============================] - 33s 819ms/step - loss: 0.8900 - accuracy: 0.8035 - val_loss: 0.2556 - val_accuracy: 0.9684
Epoch 2/10
36/36 [==============================] - 29s 819ms/step - loss: 0.4682 - accuracy: 0.8675 - val_loss: 0.0359 - val_accuracy: 0.9926
Epoch 3/10
36/36 [==============================] - 29s 806ms/step - loss: 0.3858 - accuracy: 0.8852 - val_loss: 0.0462 - val_accuracy: 0.9857
Epoch 4/10
36/36 [==============================] - 29s 808ms/step - loss: 0.3426 - accuracy: 0.8974 - val_loss: 0.0751 - val_accuracy: 0.9824
Epoch 5/10
36/36 [==============================] - 29s 805ms/step - loss: 0.3407 - accuracy: 0.9011 - val_loss: 0.1352 - val_accuracy: 0.9737
Epoch 6/10
36/36 [==============================] - 29s 805ms/step - loss: 0.2467 - accuracy: 0.9391 - val_loss: 0.2548 - val_accuracy: 0.9599
Epoch 7/10
36/36 [==============================] - 29s 813ms/step - loss: 0.2588 - accuracy: 0.9264 - val_loss: 0.1776 - val_accuracy: 0.9

In [37]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

12
Epoch 1/10
39/39 [==============================] - 34s 793ms/step - loss: 0.7763 - accuracy: 0.8198 - val_loss: 0.0624 - val_accuracy: 0.9844
Epoch 2/10
39/39 [==============================] - 30s 786ms/step - loss: 0.4571 - accuracy: 0.8682 - val_loss: 0.0575 - val_accuracy: 0.9862
Epoch 3/10
39/39 [==============================] - 31s 789ms/step - loss: 0.4160 - accuracy: 0.8885 - val_loss: 0.0749 - val_accuracy: 0.9819
Epoch 4/10
39/39 [==============================] - 31s 790ms/step - loss: 0.3657 - accuracy: 0.8964 - val_loss: 0.0706 - val_accuracy: 0.9842
Epoch 5/10
39/39 [==============================] - 31s 788ms/step - loss: 0.3328 - accuracy: 0.9105 - val_loss: 0.1074 - val_accuracy: 0.9804
Epoch 6/10
39/39 [==============================] - 31s 788ms/step - loss: 0.2541 - accuracy: 0.9305 - val_loss: 0.1386 - val_accuracy: 0.9709
Epoch 7/10
39/39 [==============================] - 32s 830ms/step - loss: 0.2536 - accuracy: 0.9252 - val_loss: 0.2155 - val_accuracy: 0.9

In [38]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

13
Epoch 1/10
42/42 [==============================] - 36s 775ms/step - loss: 0.9619 - accuracy: 0.7873 - val_loss: 0.0663 - val_accuracy: 0.9880
Epoch 2/10
42/42 [==============================] - 32s 771ms/step - loss: 0.5404 - accuracy: 0.8508 - val_loss: 0.0418 - val_accuracy: 0.9867
Epoch 3/10
42/42 [==============================] - 32s 771ms/step - loss: 0.4516 - accuracy: 0.8636 - val_loss: 0.0411 - val_accuracy: 0.9890
Epoch 4/10
42/42 [==============================] - 33s 783ms/step - loss: 0.3800 - accuracy: 0.8955 - val_loss: 0.0577 - val_accuracy: 0.9862
Epoch 5/10
42/42 [==============================] - 32s 770ms/step - loss: 0.3256 - accuracy: 0.9105 - val_loss: 0.1366 - val_accuracy: 0.9770
Epoch 6/10
42/42 [==============================] - 32s 775ms/step - loss: 0.3368 - accuracy: 0.9128 - val_loss: 0.0924 - val_accuracy: 0.9752
Epoch 7/10
42/42 [==============================] - 32s 774ms/step - loss: 0.2658 - accuracy: 0.9241 - val_loss: 0.1562 - val_accuracy: 0.9

In [39]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

14
Epoch 1/10
45/45 [==============================] - 30s 584ms/step - loss: 0.9124 - accuracy: 0.8016 - val_loss: 0.1897 - val_accuracy: 0.9758
Epoch 2/10
45/45 [==============================] - 26s 587ms/step - loss: 0.5703 - accuracy: 0.8403 - val_loss: 0.0471 - val_accuracy: 0.9880
Epoch 3/10
45/45 [==============================] - 26s 585ms/step - loss: 0.4361 - accuracy: 0.8747 - val_loss: 0.0613 - val_accuracy: 0.9855
Epoch 4/10
45/45 [==============================] - 26s 588ms/step - loss: 0.3397 - accuracy: 0.9020 - val_loss: 0.0661 - val_accuracy: 0.9857
Epoch 5/10
45/45 [==============================] - 26s 585ms/step - loss: 0.2930 - accuracy: 0.9198 - val_loss: 0.1368 - val_accuracy: 0.9750
Epoch 6/10
45/45 [==============================] - 26s 584ms/step - loss: 0.3360 - accuracy: 0.9088 - val_loss: 0.1824 - val_accuracy: 0.9627
Epoch 7/10
45/45 [==============================] - 26s 585ms/step - loss: 0.2678 - accuracy: 0.9254 - val_loss: 0.2024 - val_accuracy: 0.9

In [40]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

15
Epoch 1/10
48/48 [==============================] - 33s 599ms/step - loss: 0.9463 - accuracy: 0.7851 - val_loss: 0.1464 - val_accuracy: 0.9829
Epoch 2/10
48/48 [==============================] - 28s 578ms/step - loss: 0.5318 - accuracy: 0.8505 - val_loss: 0.0488 - val_accuracy: 0.9878
Epoch 3/10
48/48 [==============================] - 28s 579ms/step - loss: 0.4061 - accuracy: 0.8816 - val_loss: 0.0441 - val_accuracy: 0.9880
Epoch 4/10
48/48 [==============================] - 27s 573ms/step - loss: 0.3764 - accuracy: 0.8937 - val_loss: 0.0562 - val_accuracy: 0.9862
Epoch 5/10
48/48 [==============================] - 27s 574ms/step - loss: 0.3279 - accuracy: 0.9039 - val_loss: 0.0728 - val_accuracy: 0.9798
Epoch 6/10
48/48 [==============================] - 27s 576ms/step - loss: 0.3114 - accuracy: 0.9147 - val_loss: 0.1877 - val_accuracy: 0.9673
Epoch 7/10
48/48 [==============================] - 28s 577ms/step - loss: 0.2524 - accuracy: 0.9307 - val_loss: 0.1843 - val_accuracy: 0.9

In [41]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

16
Epoch 1/10
51/51 [==============================] - 32s 570ms/step - loss: 1.0400 - accuracy: 0.7616 - val_loss: 0.0416 - val_accuracy: 0.9906
Epoch 2/10
51/51 [==============================] - 29s 570ms/step - loss: 0.5605 - accuracy: 0.8453 - val_loss: 0.0558 - val_accuracy: 0.9865
Epoch 3/10
51/51 [==============================] - 29s 569ms/step - loss: 0.4089 - accuracy: 0.8824 - val_loss: 0.1065 - val_accuracy: 0.9788
Epoch 4/10
51/51 [==============================] - 29s 567ms/step - loss: 0.3889 - accuracy: 0.8923 - val_loss: 0.1241 - val_accuracy: 0.9801
Epoch 5/10
51/51 [==============================] - 29s 566ms/step - loss: 0.3124 - accuracy: 0.9122 - val_loss: 0.1323 - val_accuracy: 0.9742
Epoch 6/10
51/51 [==============================] - 29s 578ms/step - loss: 0.3018 - accuracy: 0.9157 - val_loss: 0.2070 - val_accuracy: 0.9750
Epoch 7/10
51/51 [==============================] - 29s 566ms/step - loss: 0.2474 - accuracy: 0.9269 - val_loss: 0.3151 - val_accuracy: 0.9

In [42]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

17
Epoch 1/10
54/54 [==============================] - 41s 712ms/step - loss: 1.1313 - accuracy: 0.7501 - val_loss: 0.0463 - val_accuracy: 0.9900
Epoch 2/10
54/54 [==============================] - 52s 964ms/step - loss: 0.5782 - accuracy: 0.8444 - val_loss: 0.0313 - val_accuracy: 0.9913
Epoch 3/10
54/54 [==============================] - 43s 807ms/step - loss: 0.5010 - accuracy: 0.8595 - val_loss: 0.0477 - val_accuracy: 0.9865
Epoch 4/10
54/54 [==============================] - 39s 724ms/step - loss: 0.4275 - accuracy: 0.8755 - val_loss: 0.3423 - val_accuracy: 0.9232
Epoch 5/10
54/54 [==============================] - 39s 728ms/step - loss: 0.4065 - accuracy: 0.8845 - val_loss: 0.0595 - val_accuracy: 0.9870
Epoch 6/10
54/54 [==============================] - 39s 731ms/step - loss: 0.3081 - accuracy: 0.9114 - val_loss: 0.0601 - val_accuracy: 0.9862
Epoch 7/10
54/54 [==============================] - 39s 724ms/step - loss: 0.3019 - accuracy: 0.9117 - val_loss: 0.0381 - val_accuracy: 0.9

In [43]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

18
Epoch 1/10
57/57 [==============================] - 44s 718ms/step - loss: 1.1197 - accuracy: 0.7353 - val_loss: 0.0663 - val_accuracy: 0.9875
Epoch 2/10
57/57 [==============================] - 41s 718ms/step - loss: 0.6202 - accuracy: 0.8326 - val_loss: 0.0331 - val_accuracy: 0.9936
Epoch 3/10
57/57 [==============================] - 41s 716ms/step - loss: 0.4851 - accuracy: 0.8671 - val_loss: 0.0394 - val_accuracy: 0.9898
Epoch 4/10
57/57 [==============================] - 42s 732ms/step - loss: 0.4159 - accuracy: 0.8749 - val_loss: 0.0367 - val_accuracy: 0.9913
Epoch 5/10
57/57 [==============================] - 41s 720ms/step - loss: 0.3673 - accuracy: 0.9015 - val_loss: 0.0401 - val_accuracy: 0.9908
Epoch 6/10
57/57 [==============================] - 41s 718ms/step - loss: 0.3081 - accuracy: 0.9100 - val_loss: 0.0351 - val_accuracy: 0.9923
Epoch 7/10
57/57 [==============================] - 41s 715ms/step - loss: 0.3347 - accuracy: 0.9081 - val_loss: 0.0321 - val_accuracy: 0.9

In [44]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

19
Epoch 1/10
62/62 [==============================] - 47s 707ms/step - loss: 1.1483 - accuracy: 0.7245 - val_loss: 0.0413 - val_accuracy: 0.9913
Epoch 2/10
62/62 [==============================] - 43s 696ms/step - loss: 0.6930 - accuracy: 0.8020 - val_loss: 0.0376 - val_accuracy: 0.9936
Epoch 3/10
62/62 [==============================] - 34s 556ms/step - loss: 0.5678 - accuracy: 0.8498 - val_loss: 0.0513 - val_accuracy: 0.9885
Epoch 4/10
62/62 [==============================] - 34s 546ms/step - loss: 0.4597 - accuracy: 0.8714 - val_loss: 0.1687 - val_accuracy: 0.9666
Epoch 5/10
62/62 [==============================] - 34s 548ms/step - loss: 0.4060 - accuracy: 0.8827 - val_loss: 0.0573 - val_accuracy: 0.9860
Epoch 6/10
62/62 [==============================] - 34s 549ms/step - loss: 0.3593 - accuracy: 0.8917 - val_loss: 0.1067 - val_accuracy: 0.9686
Epoch 7/10
62/62 [==============================] - 34s 546ms/step - loss: 0.3522 - accuracy: 0.8942 - val_loss: 0.0441 - val_accuracy: 0.9

In [45]:
#new_model_nc_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_nc_e1"
new_model_nc_dir  = "D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2"
i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/gtsrb/C3/e2/gtsrb_model_c3_aug_nc_e2_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1